# *Master Thesis - Lars Schiffer*


# Imports and Configurations

## API-Keys

In [ ]:
# API Keys used for FRED, Alpha Vantage and SEC
fred_api_key = ''
alpha_vantage_api_key = ''
sec_api_key = ''

## Imports

In [ ]:
# Standard Libraries
import os
import time
import datetime
from math import sqrt
from time import sleep

# Third-party Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import requests
import json
import yfinance as yf
import matplotlib.pyplot as plt
import torch
import joblib
import torch.nn.init as init
import lightgbm as lgb
import wandb
import optuna
import xgboost as xgb
import statsmodels.tsa.stattools as stattools
# import pytorch_lightning as pl
from tqdm import tqdm
from sec_api import MappingApi
from arch import arch_model
from scipy.stats import skew, kurtosis, linregress
from datetime import timedelta, datetime
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    precision_score,
    confusion_matrix,
    log_loss,
    recall_score,
    f1_score,
    accuracy_score,
    matthews_corrcoef,
)
from sklearn.model_selection import GridSearchCV, KFold, train_test_split, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint
from torch.nn.modules.activation import Sigmoid
from torch import nn
from optuna.integration import LightGBMPruningCallback


from ray import tune
from ray.tune.search.optuna import OptunaSearch
from ray.air import session

# Additional Libraries which were used on Windows due to dependency issues on Linux and conda

# import talib
# from fracdiff import Fracdiff, fdiff
# from fracdiff.sklearn import FracdiffStat
# import plotly.graph_objects as go


## Initialize Weights & Biases

In [ ]:
wandb.login()

## Configurations

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

path_default = '/home/lschiff_ext' # default path
path_data_raw = '/home/lschiff_ext/data_v3/raw' # concatenated 16k raw files
path_data_trading_daily = '/home/lschiff_ext/data_v2/trading_daily' # initial data with 16k files for stock data
path_data_strategy = '/home/lschiff_ext/data_v5/strategy' # applied features + trading strategy
path_data_features = '/home/lschiff_ext/data_v2/features' # applied features
path_data_external = '/home/lschiff_ext/data_v3/external' # external apis
path_data_combined = '/home/lschiff_ext/data_v5/combined' # combined data from strategy and external features
path_data_buy_hold = '/home/lschiff_ext/data_v5/buy_and_hold' # combined data from strategy and external features
path_data_features_json_tbm = '/home/lschiff_ext/data_v2/features/v1_features_tbm.json' # applied features
path_data_features_json_bb = '/home/lschiff_ext/data_v2/features/v1_features_bb.json' # applied features
path_data_final_models = '/home/lschiff_ext/data_v5/predictions/models'


# final data used for training
# TBM with fees
path_data_partitioned_tbm_with_fees_train = '/home/lschiff_ext/data_v5/partitioned/with_fees/tbm/train'
path_data_partitioned_tbm_with_fees_test = '/home/lschiff_ext/data_v5/partitioned/with_fees/tbm/test'
path_data_partitioned_tbm_with_fees_pca = '/home/lschiff_ext/data_v5/partitioned/with_fees/tbm/pca'
path_data_partitioned_tbm_with_fees_scaled = '/home/lschiff_ext/data_v5/partitioned/with_fees/tbm/scaled'

# BB with fees
path_data_partitioned_bb_with_fees_train = '/home/lschiff_ext/data_v5/partitioned/with_fees/bb/train'
path_data_partitioned_bb_with_fees_test = '/home/lschiff_ext/data_v5/partitioned/with_fees/bb/test'
path_data_partitioned_bb_with_fees_pca = '/home/lschiff_ext/data_v5/partitioned/with_fees/bb/pca'
path_data_partitioned_bb_with_fees_scaled ='/home/lschiff_ext/data_v5/partitioned/with_fees/bb/scaled'

# Feature Selection
path_data_features_rfe_bb_xgboost = '/home/lschiff_ext/data_v2/features/v1_features_bb_RFE_XGBoost.json'
path_data_features_rfe_bb_lgbm = '/home/lschiff_ext/data_v2/features/v2_features_bb_RFE_LGBM_w_vert_barrier.json'
path_data_features_rfe_tbm_scaled_rob_lgbm = '/home/lschiff_ext/data_v2/features/v1_features_tbm_RFE_LGBM_Scaled_rob.json'


# Model Training Results
path_data_results_bb_rf = '/home/lschiff_ext/data_v2/results/bb/random_forest'
path_data_results_tbm_rf = '/home/lschiff_ext/data_v2/results/tbm/random_forest'

path_data_results_bb_nn = '/home/lschiff_ext/data_v3/results/bb/neural_network'
path_data_results_tbm_nn = '/home/lschiff_ext/data_v3/results/tbm/neural_network'

path_data_results_bb_log = '/home/lschiff_ext/data_v2/results/bb/logistic_regression'
path_data_results_tbm_log = '/home/lschiff_ext/data_v2/results/tbm/logistic_regression'

path_data_results_bb_xgb = '/home/lschiff_ext/data_v2/results/bb/xgboost'
path_data_results_tbm_xgb = '/home/lschiff_ext/data_v2/results/tbm/xgboost'

hold_out_date = pd.to_datetime('2020-01-01')

# General Helper Methods

In [ ]:
def print_df_rows(df, end = False, no_rows = 1):
    """
    Print rows of a DataFrame to get insights into the data.

    Args:
        df (dataframe): The DataFrame to print rows from.
        end (boolean, optional): If True, print rows from the end of the DataFrame. 
                              If False, print rows from the beginning. 
                              Default is set to False.
        no_rows (integer, optional): The number of rows to print. Default is set to 1.
    """
    # Get the specified number of rows from the DataFrame
    if end:
        idx_row = df.tail(no_rows)
    else:
        idx_row = df.head(no_rows)
    
    # Print each row
    for index, row in idx_row.iterrows():
        print(f"{index}" + '-'*30)
        for col_name in idx_row.columns:
            print(f"{col_name}: {row[col_name]}")

In [ ]:
def select_columns(df: pd.DataFrame, columns: list):
    """
    Selects specific columns from a dataframe.
    
    Args:
        df (dataframe): The input dataframe.
        columns (list): The list of column names to select.
        
    Returns:
        Dataframe: A new dataframe containing only the selected columns.
        
    Raises:
        ValueError: If any of the specified columns are missing from the dataframe.
    """
    
    # Find the missing columns
    missing_columns = set(columns) - set(df.columns)
    
    # Raise an error if there are missing columns
    if missing_columns:
        raise ValueError(f"The following columns are missing from the dataframe: {missing_columns}")
    
    # Select the specified columns and create a copy of the dataframe
    return df.loc[:, columns].copy()

In [ ]:
def retrieve_stock_data(asset, start_date, end_date, save = False):
    """
    Retrieves stock data for a given share, start date, and end date.
    
    Args:
        asset (string): The stock symbol (CUSIP).
        start_date (string): The start date in 'YYYY-MM-DD' format.
        end_date (string): The end date in 'YYYY-MM-DD' format.
        save (boolean, optional): Whether to save the data to a file. Defaults to False.
    
    Returns:
        Dataframe: The stock data with columns: ['open', 'high', 'low', 'close', 'volume'].
    """
    name = f"{asset}--{start_date}--{end_date}.csv"
    location = os.path.join(path_data_raw, name)
    
    if not os.path.exists(location):
        # Data does not exist, download and save
        data = yf.download(asset, start=start_date, end=end_date)
        data.to_csv(os.path.join(path_data_raw, name))
        print('Data received from Yahoo.')
    
    data = pd.read_csv(location)
    data = data.set_index(pd.DatetimeIndex(data['Date'].values))
    data = select_columns(data, ['Open', 'High', 'Low', 'Adj Close', 'Volume'])
    data = data.rename(columns={'Open': 'open', 'High': 'high', 'Adj Close': 'close', 'Low' : 'low', 'Volume' : 'volume'})
    
    return data

In [ ]:
def select_features_from_json(X_features, path):
    """
    Selects the specified features from a JSON file and returns the filtered dataframe.

    Args:
        X_features (dataframe): The input dataframe.
        path (string): The path to the JSON file.

    Returns:
        Dataframe: The filtered dataframe containing only the specified features.
    """

    # Load the JSON file as a dictionary
    with open(path) as f:
        features_dict = json.load(f)
        
    # Create a list of column names to keep
    columns_to_keep = [k for k, v in features_dict.items() if v == 1]

    # Filter the dataframe based on the columns to keep
    filtered_df = X_features[columns_to_keep]

    return filtered_df

In [ ]:
def read_data_csv():
    """
    Reads data from a CSV file and returns a Pandas DataFrame.

    Returns:
        Dataframe: The DataFrame containing the data from the CSV file.
    """
    # Define the data type dictionary for the columns
    dtype_dict = {
        'cusip': str,
        'open': float,
        'high': float,
        'low': float,
        'close': float,
        'volume': float
    }

    # Read the CSV file into a DataFrame
    data = pd.read_csv('/home/lschiff_ext/data_v2/raw/v1_trading_combined.csv', dtype=dtype_dict, parse_dates=['date'])

    return data

In [ ]:
def read_data_processed_csv(full_path, prefix = 'feature_'):
    """
    Reads a CSV file and processes the data.

    Args:
        full_path (string): The full path to the CSV file.
        prefix (string, optional): The prefix to be added to the column names. Defaults to 'feature_'.

    Returns:
        Dataframe: The processed data.
    """
    # Define the data types for the columns
    dtype_dict = {
        'cusip': object,
        'open': float,
        'high': float,
        'low': float,
        'close': float,
        'volume': float,
        'bin': int
    }

    # Read the CSV file
    data = pd.read_csv(full_path, dtype=dtype_dict, parse_dates=['date', 'vertical_barrier', 't1'])

    # Format the date columns
    data['date'] = data['date'].dt.strftime('%Y-%m-%d')
    data['vertical_barrier'] = data['vertical_barrier'].dt.strftime('%Y-%m-%d')
    data['t1'] = data['t1'].dt.strftime('%Y-%m-%d')

    # Convert the date columns to datetime
    data['date'] = pd.to_datetime(data['date'])
    data['vertical_barrier'] = pd.to_datetime(data['vertical_barrier'])
    data['t1'] = pd.to_datetime(data['t1'])

    # Add OHLC data as features, add prefix
    new_columns = {col: prefix + col for col in ['open', 'high', 'low', 'close']}
    data = data.rename(columns=new_columns)

    return data

In [ ]:
def read_data_raw_csv(full_path):
    """
    Read data from a CSV file and return a DataFrame.

    Args:
        full_path (string): The full path to the CSV file.

    Returns:
        Dataframe: A DataFrame containing the data from the CSV file.
    """
    # Define the data types for each column
    dtype_dict = {
        'cusip': str,
        'open': float,
        'high': float,
        'low': float,
        'close': float,
        'volume': float,
    }

    # Read the CSV file into a DataFrame
    data = pd.read_csv(full_path, dtype=dtype_dict, parse_dates=['date'])

    # Format the date column as 'YYYY-MM-DD'
    data['date'] = data['date'].dt.strftime('%Y-%m-%d')

    # Convert the date column to datetime
    data['date'] = pd.to_datetime(data['date'])

    return data

In [ ]:
def load_ml_data(path, file_name, vertical_barrier = True) -> pd.DataFrame:
    """
    Load ML data from a file.

    Args:
        path (string): The path to the directory containing the file.
        file_name (string): The name of the file to load.
        vertical_barrier (boolean, optional): Whether to include the vertical barrier column. Defaults to True.

    Returns:
        Dataframe: The loaded ML data.
    """

    # Define the data types for the columns
    dtype_dict = {
        'cusip': str,
        'bin': int,
        'side': int,
        'profit_abs': np.float32,
        't1_price': np.float32,
    }

    # Read the CSV file into a DataFrame
    if vertical_barrier:
        data = pd.read_csv(path + '/' + file_name, dtype=dtype_dict, parse_dates=['date', 't1', 'vertical_barrier'])
    else:
        data = pd.read_csv(path + '/' + file_name, dtype=dtype_dict, parse_dates=['date', 't1'])

    # Set data type of columns starting with "feature_" to float32
    feature_cols_floats = [col for col in data.columns if col.startswith('feature_')]
    data[feature_cols_floats] = data[feature_cols_floats].astype(np.float32)

    # Set data type for features with int (which are the CDL features)
    feature_cols_int = [col for col in data.columns if col.startswith('feature_CDL')]  # all CDL features are int
    data[feature_cols_int] = data[feature_cols_int].astype(int)

    return data

# Feature Engineering

## Fractional Differentiation

In [ ]:
# Here the fractional differentiation is tested
# The following lines of code are not directly related to the feature engineering
# Was used to get an idea of the fractional differentiation

# define Apple stock and timeframe
share = "AAPL"
start_date = "2005-01-01"
end_date = "2023-01-01"

# Retrieve stock_data
data = retrieve_stock_data(share, start_date, end_date, True)

# visualize
plt.figure(figsize=(24, 24))
plt.tight_layout()
plt.subplots_adjust(hspace=0.4)

for i, d in enumerate(np.linspace(0.1, 0.9, 9)):
    diff = fdiff(data['close'], d)
    diff = pd.Series(diff, index=data.index[-diff.size :])
    plt.subplot(9, 1, i + 1)
    plt.title(f"Stock Portfolio, {d:.1f}th differentiated")
    plt.plot(diff, linewidth=0.4)

plt.show()

In [ ]:
# Stationary of fractional differentiation
def adfstat(d: float) -> float:
    diff = fdiff(data['close'], d)
    stat, *_ = stattools.adfuller(diff)
    return stat

def correlation(d: float) -> np.ndarray:
    diff = fdiff(data['close'], d)
    return np.corrcoef(data['close'][-diff.size :], diff)[0, 1]

ds = np.linspace(0.0, 1.0, 10)
stats = np.vectorize(adfstat)(ds)
corrs = np.vectorize(correlation)(ds)

# 5% critical value of stationarity
_, _, _, _, crit, _ = stattools.adfuller(data['close'])

# plot
fig, ax_stat = plt.subplots(figsize=(24, 8))
ax_corr = ax_stat.twinx()

ax_stat.plot(ds, stats, color="blue", label="ADF statistics (left)")
ax_corr.plot(ds, corrs, color="orange", label="correlation (right)")
ax_stat.axhline(y=crit["5%"], linestyle="--", color="k", label="5% critical value")

plt.title("Stationarity and memory of fractionally differentiated Stock")
fig.legend()
plt.show()

In [ ]:
X = data['close'].values.reshape(-1, 1)

fs = FracdiffStat()

Xdiff = fs.fit_transform(X)
_, pvalue, _, _, _, _ = stattools.adfuller(Xdiff.reshape(-1))
corr = np.corrcoef(X[-Xdiff.size :, 0], Xdiff.reshape(-1))[0][1]

print("* Order: {:.2f}".format(fs.d_[0]))
print("* ADF p-value: {:.2f} %".format(100 * pvalue))
print("* Correlation with the original time-series: {:.2f}".format(corr))

In [ ]:
data['frac_diff'] = fdiff(data['close'], fs.d_[0])

In [ ]:
data['frac_diff'].plot()

## FRED® API (Federal Reserve Bank of St. Louis) & yFinance API

In [ ]:
def request_fred_api(series_id, start_date, end_date):
    """
    Requests data from the FRED API for a given series ID and date range.
    
    Args:
        series_id (string): The series ID to fetch data for.
        start_date (string): The start date of the date range.
        end_date (string): The end date of the date range.
    
    Returns:
        Dataframe: The fetched data as a pandas DataFrame.
    """
    # Construct the API URL
    url = f'https://api.stlouisfed.org/fred/series/observations'
    url += f'?series_id={series_id}&api_key={fred_api_key}'
    url += f'&file_type=json&observation_start={start_date}&observation_end={end_date}'
    
    # Send the API request
    response = requests.get(url)
    
    try:
        # Extract the data from the API response
        data = json.loads(response.text)['observations']
    except KeyError:
        # Handle error if data couldn't be fetched
        print(f"Error fetching data for {series_id}. Response: {response.text}")
        return pd.DataFrame()
    
    # Create a DataFrame from the fetched data
    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    df = df[['value']]
    df.columns = [series_id]
    df[series_id] = pd.to_numeric(df.iloc[:,0], errors='coerce')
    
    return df

In [ ]:
def request_yfinance_data(ticker, start_date, end_date) -> pd.DataFrame:
    """
    Requests stock data from Yahoo Finance API.

    Args:
        ticker (string): The stock ticker symbol.
        start_date (string): The start date for the data.
        end_date (string): The end date for the data.

    Returns:
        Dataframe: The stock data for the specified dates.
    """
    # Download stock data from Yahoo Finance API
    yf_data = yf.download(ticker, start=start_date, end=end_date)
    
    return yf_data

## Other Features

In [ ]:
def rolling_percentile(series, rolling_window = None):
    """
    Calculates the rolling percentile of a given series.

    Args:
        series (series): The input series (most often a given feature).
        rolling_window (integer, optional): The size of the rolling window. Defaults to None.

    Returns:
        Series: The rolling percentile values.
    """
    if rolling_window is None:
        # Calculate the expanding percentile
        percentiles = series.expanding().apply(lambda x: pd.Series(x).rank(pct=True).iloc[-1])
    else:
        # Calculate the rolling percentile
        percentiles = series.rolling(window=rolling_window, min_periods=1).apply(lambda x: pd.Series(x).rank(pct=True).iloc[-1])
    
    return percentiles

In [ ]:
def compute_percentile_and_change(data, feature, rolling_window = None):
    """
    Compute the rolling percentile and percentage change for the given feature in the data.

    Args:
        data (dataframe): The input data (FRED and yFinance requested data)
        feature (string): The name of the feature to compute the percentile and change for.
        rolling_window (integer, optional): The size of the rolling window. Defaults to None.

    Returns:
        Dataframe: The input data with additional columns for the rolling percentile and percentage change.
    """

    # Calculate rolling percentile for the entire DataFrame
    percentile_df = data[feature].transform(lambda x: rolling_percentile(x, rolling_window)).to_frame(f'{feature}_rolling_percentile')

    # Calculate percentage change for the entire DataFrame
    change_df = data[feature].transform(lambda x: x.pct_change()).to_frame(f'{feature}_change')

    # Combine the original data with the change and percentile dataframes
    combined = pd.concat([data, change_df, percentile_df], axis=1)

    return combined

In [ ]:
def WMA(series, period):
    """
    Weighted Moving Average (WMA) function.

    Args:
        series (series): The input series.
        period (integer): The number of periods to calculate the weighted moving average.

    Returns:
        Series: The weighted moving average of the input series.
    """
    # Create an array of weights from 1 to period
    weights = np.arange(1, period + 1)
    
    # Calculate the weighted moving average using rolling window and dot product
    wma = series.rolling(window=period).apply(lambda x: np.dot(x, weights) / weights.sum(), raw=True)
    
    return wma

In [ ]:
def HMA(series, period):
    """
    Calculates the Hull Moving Average (HMA) of a series.

    Args:
        series (series): The input series.
        period (integer): The period for calculating the HMA.

    Returns:
        Series: The calculated HMA value.
    """
    # Calculate the half period
    half_period = int(period / 2)
    
    # Calculate the weighted moving average for the first half period
    wma1 = WMA(series, half_period)
    
    # Calculate the weighted moving average for the full period
    wma2 = WMA(series, period)
    
    # Calculate the weighted moving average of the difference between 2*wma1 and wma2
    hma = WMA(2*wma1 - wma2, int(np.sqrt(period)))
    
    # Return the calculated HMA value
    return hma

In [ ]:
def fit_garch_and_get_volatility(series):
    """
    Fits a GARCH model to a time series and returns the conditional volatility.

    Args:
        series (series): The time series data (here: the closing prices of the asset).

    Returns:
        Series: The conditional volatility of the time series.
    """
    # Define the number of lag variances to include in the GARCH model.
    p = 1
    # Define the number of lag residual errors to include in the GARCH model.
    q = 1
    
    # Create the GARCH model with the specified parameters.
    model = arch_model(series, vol='Garch', p=p, q=q)
    
    # Fit the model to the data.
    results = model.fit(update_freq=0, disp='off')
    
    # Get the conditional volatility from the fitted model.
    volatility = results.conditional_volatility
    
    # Return the conditional volatility.
    return volatility

In [ ]:
def z_score(series, window):
    """
    Calculate the z-score of a series using a rolling window.
    
    Args:
        series (series): The input series (here: the closing prices of the asset).
        window (integer): The size of the rolling window.
    
    Returns:
        Series: The z-score of the input series.
    """
    # Calculate the rolling mean with a shifted window
    rolling = series.rolling(window=window)
    rolling_mean = rolling.mean().shift(1)
    
    # Calculate the rolling standard deviation with a shifted window
    std_dev = rolling.std(ddof=0).shift(1)
    
    # Calculate the z-score
    z = (series - rolling_mean) / std_dev
    
    return z

In [ ]:
def rolling_hurst(series):
    """
    Calculates the Hurst exponent for a given time series using a rolling window approach.
    
    Args:
        series (series): The input time series data (here: applied to closing prices of the asset).
        
    Returns:
        float: The calculated Hurst exponent.
    """
    # Calculate log returns
    log_ret = np.log(series).diff()

    # Define the range of lags
    lags = range(2, 9)  # Use a smaller range of lags, e.g., 2 to 8

    # Calculate the variance of log returns for each lag
    variances = [log_ret.shift(lag).dropna().var() for lag in lags]

    # Perform linear regression on the log-log plot of variances
    regression = linregress(np.log10(lags), np.log10(variances))

    # Calculate the Hurst exponent (slope of the regression line)
    hurst_exponent = regression[0] / 2

    return hurst_exponent

In [ ]:
def calculate_dpo(data, period, min_period):
    """
    Calculate the Detrended Price Oscillator (DPO) for a given data series.

    Args:
        data (series): The input data series (here: the closing prices of the asset).
        period (integer): The number of periods to consider for the moving average.
        min_period (integer): The minimum number of periods required to calculate the moving average.

    Returns:
        Series: The calculated Detrended Price Oscillator (DPO) series.
    """
    # Shift the price series by half the period + 1
    shifted_price = data.shift(period // 2 + 1)

    # Calculate the simple moving average using the given period and min_period
    simple_moving_average = data.rolling(window=period, min_periods=min_period).mean()

    # Calculate the Detrended Price Oscillator by subtracting the simple moving average from the shifted price
    dpo = shifted_price - simple_moving_average

    return dpo

In [ ]:
def add_certain_ta_lib_features(df, prefix = 'feature_'):
    """
    Add certain technical analysis features to the given dataframe.
    
    Args:
        df (dataframe): The input dataframe (here: the data of one asset).
        prefix (string, optional): The prefix to be added to the feature names. Defaults to 'feature_'.
        
    Returns:
        Dataframe: The dataframe with the added technical analysis features.
    """
    
    # Create a dataframe to store the technical features
    df_technical_features = pd.DataFrame()
    
    # Calculate log return
    df[prefix + 'log_return'] = np.log(df['close'] / df['close'].shift(1))
    
    # Calculate various Ta-Lib features
    df_technical_features[prefix + 'BOP'] = talib.BOP(df['open'], df['high'], df['low'], df['close'])
    df_technical_features[prefix + 'NATR'] = talib.NATR(df['high'], df['low'], df['close'], 4)
    df_technical_features[prefix + 'WMA'] = talib.WMA(df['close'], 4)
    df_technical_features[prefix + 'DEMA'] = talib.DEMA(df['close'], 4)
    df_technical_features[prefix + 'TEMA'] = talib.TEMA(df['close'], 4)
    df_technical_features[prefix + 'PROC'] = talib.ROC(df['close'], 4) / df['close'].shift(4)
    df_technical_features[prefix + 'ADX'] = talib.ADX(df['high'], df['low'], df['close'], 4)
    df_technical_features[prefix + 'ADX2'] = talib.ADX(df['high'], df['low'], df['close'], 6)
    df_technical_features[prefix + 'ADX3'] = talib.ADX(df['high'], df['low'], df['close'], 8)
    df_technical_features[prefix + 'ATR'] = talib.ATR(df['high'], df['low'], df['close'], 4)
    df_technical_features[prefix + 'NATR'] = talib.NATR(df['high'], df['low'], df['close'], 4)
    df_technical_features[prefix + 'BETA1'] = talib.BETA(df['high'], df['low'], 4)
    df_technical_features[prefix + 'BETA2'] = talib.BETA(df['high'], df['low'], 6)
    df_technical_features[prefix + 'BETA3'] = talib.BETA(df['high'], df['low'], 8)
    df_technical_features[prefix + 'MOM'] = talib.MOM(df['close'], 4)
    df_technical_features[prefix + 'KAMA'] = talib.KAMA(df['close'], 4)
    df_technical_features[prefix + 'SAR'] = talib.SAR(df['high'], df['low'])
    df_technical_features[prefix + 'ROC'] = talib.ROC(df['close'], 4)
    df_technical_features[prefix + 'RSI1'] = talib.RSI(df['close'], 4)
    df_technical_features[prefix + 'RSI2'] = talib.RSI(df['close'], 8)
    df_technical_features[prefix + 'CCI'] = talib.CCI(df['high'], df['low'], df['close'], timeperiod=14)
    
    # Pattern Recognition
    df_technical_features[prefix + 'CDLSHOOTINGSTAR'] = talib.CDLSHOOTINGSTAR(df['open'], df['high'], df['low'], df['close']) # Shooting Star
    df_technical_features[prefix + 'CDLRISEFALL3METHODS'] = talib.CDLRISEFALL3METHODS(df['open'], df['high'], df['low'], df['close']) # Rising/Falling Three Methods
    df_technical_features[prefix + 'CDLINVERTEDHAMMER'] = talib.CDLINVERTEDHAMMER(df['open'], df['high'], df['low'], df['close']) # Inverted Hammer
    df_technical_features[prefix + 'CDLHARAMI'] = talib.CDLHARAMI(df['open'], df['high'], df['low'], df['close']) # Harami Patterns
    df_technical_features[prefix + 'CDLHAMMER'] = talib.CDLHAMMER(df['open'], df['high'], df['low'], df['close']) # Hammer
    df_technical_features[prefix + 'CDLGRAVESTONEDOJI'] = talib.CDLGRAVESTONEDOJI(df['open'], df['high'], df['low'], df['close']) # Gravestone Doji
    df_technical_features[prefix + 'CDLDOJI'] = talib.CDLDOJI(df['open'], df['high'], df['low'], df['close']) # Doji
    df_technical_features[prefix + 'CDL3WHITESOLDIERS'] = talib.CDL3WHITESOLDIERS(df['open'], df['high'], df['low'], df['close']) # Three White Soldiers

    df_technical_features[prefix + 'min_ret'] = df['close'].pct_change().rolling(window=8).min()
    df_technical_features[prefix + 'max_ret'] = df['close'].pct_change().rolling(window=8).max()

    # Autocorrelation
    autocorrelation_window = 20
    df_technical_features[prefix + 'autocorr_1'] = df['close'].rolling(window=autocorrelation_window, min_periods=4, center=False).apply(lambda x: x.autocorr(lag=1), raw=False)
    df_technical_features[prefix + 'autocorr_2'] = df['close'].rolling(window=autocorrelation_window, min_periods=4, center=False).apply(lambda x: x.autocorr(lag=2), raw=False)

    # Additional Features
    df_technical_features[prefix + 'EMA'] = talib.EMA(df['close'], 4) # Exponential Moving Average, period=4
    df_technical_features[prefix + 'WMA'] = talib.WMA(df['close'], 4) # Weighted Moving Average, period=4
    df_technical_features[prefix + 'DEMA'] = talib.DEMA(df['close'], 4) # Double Exponential Moving Average, period=4
    df_technical_features[prefix + 'TEMA'] = talib.TEMA(df['close'], 4) # Triple Exponential Moving Average, period=4
    df_technical_features[prefix + 'PROC'] = talib.ROC(df['close'], 4) / df['close'].shift(4) # Price Rate of Change
    df_technical_features[prefix + 'stoch_slowk'], df_technical_features[prefix + 'stoch_slowd'] = talib.STOCH(df['high'], df['low'], df['close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0) # Stochastic

    df_technical_features[prefix + 'AROON_UP'], df_technical_features[prefix + 'AROON_DOWN'] = talib.AROON(df['high'], df['low'], timeperiod=14) #Aroon
    df_technical_features[prefix + 'CORREL'] = talib.CORREL(df['high'], df['low'], 4) # Pearson's Correlation Coefficient (r)
    df_technical_features[prefix + 'HT_DCPERIOD'] = talib.HT_DCPERIOD(df['close']) # Hilbert Transform - Dominant Cycle Period
    df_technical_features[prefix + 'HT_DCPHASE'] = talib.HT_DCPHASE(df['close']) # Hilbert Transform - Dominant Cycle Phase
    df_technical_features[prefix + 'HT_TRENDMODE'] = talib.HT_TRENDMODE(df['close']) # Hilbert Transform - Trend vs Cycle Mode

    # Weighted Moving Average Mean
    df_technical_features[prefix + 'hma_mean1'] = HMA(df[prefix + 'log_return'], 2)
    df_technical_features[prefix + 'hma_mean2'] = HMA(df[prefix + 'log_return'], 4)
    df_technical_features[prefix + 'hma_mean3'] = HMA(df[prefix + 'log_return'], 8)

    # Garch Volatility for close and log return
    df_technical_features[prefix + 'garch_vol_close'] = fit_garch_and_get_volatility(df['close'].dropna())

    # Z-score
    df_technical_features[prefix + 'z_score'] = z_score(df['close'], 8) # window=8

    # Bollinger Bands
    upper, middle, lower = talib.BBANDS(df['close'], 4)
    df_technical_features[prefix + 'bb_upper'] = upper
    df_technical_features[prefix + 'bb_middle'] = middle
    df_technical_features[prefix + 'bb_lower'] = lower

    # Hurst Exponent
    df_technical_features[prefix + 'hurst'] = df['close'].rolling(8).apply(rolling_hurst, raw=False)

    # Detrended Price Oscillator
    df_technical_features[prefix + 'dpo'] = calculate_dpo(df['close'], 8, 2) # period=8, min_period=2

    # TSF
    df_technical_features[prefix + 'TSF'] = talib.TSF(df[prefix + 'log_return'], 2)
    df_technical_features[prefix + 'TSF4'] = talib.TSF(df[prefix + 'log_return'], 4)
    df_technical_features[prefix + 'TSF8'] = talib.TSF(df[prefix + 'log_return'], 8)

    # STDDEV
    df_technical_features[prefix +'STDDEV1'] = talib.STDDEV(df[prefix + 'log_return'], 1)
    df_technical_features[prefix +'STDDEV2'] = talib.STDDEV(df[prefix + 'log_return'], 2)
    df_technical_features[prefix +'STDDEV3'] = talib.STDDEV(df[prefix + 'log_return'], 3)
    df_technical_features[prefix +'STDDEV4'] = talib.STDDEV(df[prefix + 'log_return'], 4)
    df_technical_features[prefix +'STDDEV6'] = talib.STDDEV(df[prefix +'log_return'], 6)
    df_technical_features[prefix +'STDDEV8'] = talib.STDDEV(df[prefix +'log_return'], 8)

    df[prefix + 'lag_target'] = df.close.pct_change().shift(1)
    df[prefix + 'std'] = df[prefix + 'lag_target'].rolling(3).std()

    '''
    # done on Windows PC as Conda had issues with the dependency
    # Fractional Differentiation
    df = fractional_differentiation(df)
    '''

    data = pd.concat([df, df_technical_features], axis=1)
    
    return data

In [ ]:
def get_fred_data(start_date, end_date, prefix = 'feature_'):
    """
    Retrieves different data from the FRED API.

    Args:
        start_date (string): The start date of the data in the format 'YYYY-MM-DD'.
        end_date (string): The end date of the data in the format 'YYYY-MM-DD'.
        prefix (string, optional): The prefix to add to the column names. Defaults to 'feature_'.

    Returns:
        Dataframe: The retrieved data with column names prefixed with 'feature_'.
    """
    # Retrieve basic economic indicators
    gdp = request_fred_api('GDP', start_date, end_date)
    unemployment = request_fred_api('UNRATE', start_date, end_date)
    inflation = request_fred_api('T10YIE', start_date, end_date)
    m2 = request_fred_api('WM2NS', start_date, end_date)

    # Retrieve crypto data
    crypto_market_cap = request_fred_api('CBBTCUSD', start_date, end_date)

    # Combine data into a single DataFrame
    data = pd.concat([gdp.iloc[:,0],
                     unemployment.iloc[:,0],
                     inflation.iloc[:,0],
                     m2.iloc[:,0],
                     crypto_market_cap.iloc[:,0]], axis=1)

    # Set column names
    data.columns = ['GDP', 'Unemployment', 'Inflation', 'M2', 'CryptoMarketCap']
    data.columns = [prefix + col for col in data.columns]  # Add 'feature_' prefix

    # Reindex the DataFrame to include all dates in the specified range
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    data = data.reindex(date_range)

    # Fill missing values using forward fill
    data = data.fillna(method='ffill')

    return data

In [ ]:
def get_yfinance_data(start_date, end_date, prefix = 'feature_'):
    """
    Retrieves financial data from Yahoo Finance API and preprocesses it.
    
    Args:
      start_date (string): Start date of data retrieval (format: 'YYYY-MM-DD')
      end_date (string): End date of data retrieval (format: 'YYYY-MM-DD')
      prefix (string): Prefix to add to column names (default: 'feature_')
    
    Returns:
      Dataframe: Preprocessed financial data
    """
    
    # Commoditites
    gold = request_yfinance_data('GC=F', start_date, end_date)
    silver = request_yfinance_data('SI=F', start_date, end_date)
    crude_oil = request_yfinance_data('CL=F', start_date, end_date)

    # Volatility for Commoditites
    gold_returns= np.log(gold['Close'] / gold['Close'].shift(1))
    gold_vol = gold_returns.rolling(window=20).std() * np.sqrt(20) 
    silver_returns= np.log(silver['Close'] / silver['Close'].shift(1))
    silver_vol = silver_returns.rolling(window=20).std() * np.sqrt(20) 
    crude_oil_returns = np.log(crude_oil['Close'] / silver['Close'].shift(1))
    crude_oil_vol = crude_oil_returns.rolling(window=20).std() * np.sqrt(20) 

    # Equity market indices
    sp500 = request_yfinance_data('^GSPC', start_date, end_date)
    nasdaq = request_yfinance_data('^IXIC', start_date, end_date)
    dowjones = request_yfinance_data('^DJI', start_date, end_date)

    # Volatility for equity market indices
    sp500_returns = np.log(sp500['Close'] / sp500['Close'].shift(1))
    sp500_vol = sp500_returns.rolling(window=20).std() * np.sqrt(20)
    nasdaq_returns = np.log(nasdaq['Close'] / nasdaq['Close'].shift(1))
    nasdaq_vol = nasdaq_returns.rolling(window=20).std() * np.sqrt(20)
    dowjones_returns = np.log(dowjones['Close'] / dowjones['Close'].shift(1))
    dowjones_vol = dowjones_returns.rolling(window=20).std() * np.sqrt(20) 

    # Crypto
    bitcoin = request_yfinance_data('BTC-USD', start_date, end_date)

    # U.S. Dollar Index
    usd = request_yfinance_data('DX-Y.NYB', start_date, end_date)

    # Treasury yields on U.S. government-issued securities
    treasury_yield1 = request_yfinance_data('^TNX', start_date, end_date)
    treasury_yield2 = request_yfinance_data('^IRX', start_date, end_date)
    treasury_yield3 = request_yfinance_data('^FVX', start_date, end_date)
    treasury_yield4 = request_yfinance_data('^TYX', start_date, end_date)

    # Calculate yield_curve_spread
    yield_curve_spread = treasury_yield4['Close'] - treasury_yield2['Close']

    # Calculate Moving Average for "main" treasury_yield
    treasury_yield1_ma30 = treasury_yield1['Close'].rolling(window=30).mean()

    # Volatility treasury_yield
    treasury_yield1_vol = treasury_yield1['Close'].rolling(window=20).std() * np.sqrt(20) 

    # New highs, new lows
    sp500_new_highs = (sp500['Close'] > sp500['Close'].rolling(window=52).max().shift(1)).astype(int)
    sp500_new_lows = (sp500['Close'] < sp500['Close'].rolling(window=52).min().shift(1)).astype(int)

    # Concatenate data
    data = pd.concat([sp500['Close'],
                    sp500_vol,
                    nasdaq['Close'],
                    nasdaq_vol,
                    dowjones['Close'],
                    dowjones_vol,
                    gold['Close'],
                    gold_vol,
                    silver['Close'],
                    silver_vol,
                    crude_oil['Close'],
                    crude_oil_vol,
                    bitcoin['Close'],
                    usd['Close'],
                    treasury_yield1['Close'],
                    treasury_yield2['Close'],
                    treasury_yield3['Close'],
                    treasury_yield4['Close'],
                    yield_curve_spread,
                    treasury_yield1_ma30,
                    treasury_yield1_vol,
                    sp500_new_highs,
                    sp500_new_lows], axis=1)
    
    # Set column names
    data.columns = [
      'SP500',
      'SP500_Vol',
      'NASDAQ',
      'NASDAQ_Vol',
      'DowJones',
      'DowJones_Vol',
      'Gold',
      'Gold_Daily_Vol',
      'Silver',
      'Silver_Daily_Vol',
      'CrudeOil',
      'CrudeOil_Vol',
      'BTC',
      'USDIndex',
      'Treasury_Yield1',
      'Treasury_Yield2',
      'Treasury_Yield3',
      'Treasury_Yield4',
      'yield_curve_spread',
      'treasury_yield1_ma30',
      'treasury_yield1_vol',
      'SP500_new_highs',
      'SP500_new_lows'
    ]

    data.columns = [prefix + col for col in data.columns]  # Add 'feature_' prefix

    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    data = data.reindex(date_range)

    data = data.fillna(method='ffill')  # Forward fill missing data

    return data

In [ ]:
def fractional_differentiation(data: pd.DataFrame, close: str = 'close', prefix: str = 'feature_', verbose: bool = False) -> pd.DataFrame:
    """
    Apply fractional differentiation to the closing prices of the given data. Use the minimum order of fractional differentiation that makes the time series stationary.

    Args:
        data (dataframe): The input data.
        close (string, optional): The name of the 'close' column in the data. Defaults to 'close'.
        prefix (string, optional): The prefix to add to the new column name. Defaults to 'feature_'.
        verbose (boolean, optional): Whether to print verbose information. Defaults to False.

    Returns:
        Dataframe: The modified data with the new column added.
    """
    X = data[close].values.reshape(-1, 1)

    # Perform fractional differentiation
    fs = FracdiffStat()
    try:
        Xdiff = fs.fit_transform(X)
        _, pvalue, _, _, _, _ = stattools.adfuller(Xdiff.reshape(-1))
        corr = np.corrcoef(X[-Xdiff.size :, 0], Xdiff.reshape(-1))[0][1]
        data[prefix + 'frac_diff_close'] = fdiff(data[close], fs.d_[0])
    except Exception as e:
        # Set the new column to NaN if an error occurs
        data[prefix + 'frac_diff_close'] = np.nan

    if verbose:
        # Print verbose information
        print("* Order: {:.2f}".format(fs.d_[0]))
        print("* ADF p-value: {:.2f} %".format(100 * pvalue))
        print("* Correlation with the original time-series: {:.2f}".format(corr))
    
    return data

# Trading Strategies

## BB-Strategy (without vertical barrier)

In [ ]:
def calculate_bb(df, window_size = 20, num_std = 2):
    """
    Calculate the Bollinger Bands for a given DataFrame.

    Args:
        df (dataframe): DataFrame containing the closing prices.
        window_size (integer, optional): Size of the rolling window for calculating the mean and standard deviation. Default set to 20.
        num_std (integer, optional): Number of standard deviations to use for calculating the upper and lower bands. Default set to 2.

    Returns:
        Dataframe: DataFrame with the Bollinger Bands added as new columns.
    """

    # Calculate the rolling mean and standard deviation of the closing prices
    rolling_mean = df['close'].rolling(window=window_size).mean()
    rolling_std = df['close'].rolling(window=window_size).std()

    # Calculate the upper and lower Bollinger Bands
    upper_band = rolling_mean + (rolling_std * num_std)
    lower_band = rolling_mean - (rolling_std * num_std)

    # Add the calculated values to the DataFrame as new columns
    df['sma_20'] = rolling_mean
    df['upper_bb'] = upper_band
    df['lower_bb'] = lower_band

    return df

In [ ]:
def calculate_trade_signals_bb_tbm(df):
    """
    Calculate trade signals based on Bollinger Bands and Moving Average. Method is used from TBM-Strategy.
    
    Args:
        df (dataframe): The input dataframe containing 'close', 'upper_bb', 'lower_bb', and 'sma_20' columns.
        
    Returns:
        Dataframe: The modified dataframe with added 'side' column indicating long and short signals.
    """
    # Check if the columns are already in df, if not calculate them
    if ('upper_bb' not in df) or ('lower_bb' not in df) or ('sma_20' not in df):
        df = calculate_bb(df)

    df['side'] = np.nan
    long_signals = (df['close'] <= df['lower_bb'])
    short_signals = (df['close'] >= df['upper_bb'])

    df.loc[long_signals, 'side'] = 1 # long signals = 1
    df.loc[short_signals, 'side'] = -1 # short signals = -1

    # Remove Look ahead bias by lagging the signal by a day
    df['side'] = df['side'].shift(1)

    # Drop NaN values so we only have entries with -1 and 1 (long and short signals)
    df.dropna(axis=0, subset=['side'], inplace=True)
    
    return df

In [ ]:
def apply_bollinger_bands(data, fee = 0.006):
    """
    Apply Bollinger Bands trading strategy to the given data.

    Parameters:
        data (dataframe): The input data containing OHLC (Open-High-Low-Close) values.
        fee (float, optional): The transaction fee as a percentage. Default set to 0.06%

    Returns:
        Dataframe: The output data containing the trading signals, profits, and other information.
    """

    output_list = []

    # Calculate trade signals using Bollinger Bands
    data = calculate_trade_signals_bb_tbm(data)

    # Initialize variables for buying and selling values
    trade_item = pd.DataFrame()
    sell_p = np.nan
    sell_date = np.nan

    for index, item in data.iterrows():
        # Skip if there is a sell signal before a buy signal
        if item['side'] == -1 and trade_item.empty:
            continue
        # If there is a buy signal and no existing trade, start a new trade
        elif item['side'] == 1 and trade_item.empty:
            trade_item = item.to_frame().transpose()
        # If there is a buy signal but there is already a previous buy signal before a sell signal came, skip
        elif item['side'] == 1 and not trade_item.empty:
            continue
        # Sell the item again
        elif item['side'] == -1 and np.isnan(sell_p):
            sell_p = item['close']
            sell_date =  pd.to_datetime(index.strftime('%Y-%m-%d'))
            profit_rel = (sell_p - trade_item['close'].iloc[0]) / trade_item['close'].iloc[0]
            
            trade_item['profit_rel'] = profit_rel
            
            # Add fee to the profits if fee is greater than 0
            if fee > 0:
                profit_rel = trade_item['profit_rel'] - fee
                trade_item['profit_rel'] = profit_rel
            
            trade_item['bin'] = 1 if profit_rel > 0 else 0
            trade_item['t1'] = sell_date
            trade_item['t1_price'] = sell_p

            output_list.append(trade_item)

            # Reset variables for the next trade
            trade_item = pd.DataFrame()
            sell_p = np.nan
            sell_date = np.nan

    try:
        output = pd.concat(output_list, axis=0)
    except Exception as e:
        output = pd.DataFrame()

    return output

## BB-Strategy (with vertical barrier)

In [ ]:
def get_vertical_barrier_item(trading_daily, trading_date, vertical_barrier_days, max_search_days = 7):
    """
    Get the first row in the trading_daily DataFrame with a date that falls within the vertical barrier.
    
    Parameters:
        trading_daily (dataframe): The DataFrame containing all OHLC data for an asset
        trading_date (timestamp): The initial trading date.
        vertical_barrier_days (integer): The number of days the barrier extends from the trading date.
        max_search_days (integer, optional): The maximum number of days to search for a matching row. Defaults to 7.
    
    Returns:
        Series: The matching row with the closest date to the vertical barrier (either on the day or the next day after the vertical barrier)
    """
    exit_date = trading_date + timedelta(days=vertical_barrier_days)

    # Search for the next available data point (necessary due to non-trading days)
    for i in range(max_search_days + 1):
        current_date = exit_date + timedelta(days=i)
        matching_rows = trading_daily[trading_daily['date'] == current_date]
        if not matching_rows.empty: # If a matching row is found
            break
        if i == max_search_days: # If no matching row is found take the last row available
            matching_rows = trading_daily.nlargest(1, 'date')
    
    matching_rows = matching_rows.set_index('date')

    return matching_rows.iloc[0]

In [ ]:
def check_if_vertical_barrier_interferes(buying_item, selling_item, vertical_barrier_days):
    """
    Check if a vertical barrier interferes between a buying item and a selling item.

    Parameters:
        buying_item (series): The buying item.
        selling_item (series): The selling item.
        vertical_barrier_days (integer): The number of days for the vertical barrier.

    Returns:
        Boolean: True if the vertical barrier interferes, False otherwise.
    """
    # Get the buying date from the buying item
    buying_date = buying_item.index[0].strftime('%Y-%m-%d')
    buying_date = pd.to_datetime(buying_date)

    # Get the possible selling date from the selling item
    possible_selling_date = selling_item.name

    # Check if the buying date plus the vertical barrier is before the selling item date
    if buying_date + timedelta(days=vertical_barrier_days) < possible_selling_date:
        return True
    else: # selling item before vertical barrier
        return False

In [ ]:
def add_selling_data(trade_item, selling_item, fee):
    """
    Adds selling data to the trade (trade_item).

    Parameters:
        trade_item (dataframe): DataFrame containing trade data.
        selling_item (series): Series containing selling data.
        fee (float): Fee as a percentage.

    Returns:
        Dataframe: Updated trade_item DataFrame.
    """
    # Get selling price and date
    sell_p = selling_item['close']
    sell_date = pd.to_datetime(selling_item.name)

    # Calculate relative profit
    profit_rel = (sell_p - trade_item['close'].iloc[0]) / trade_item['close'].iloc[0]

    # Add relative profit to trade_item DataFrame
    trade_item['profit_rel'] = profit_rel

    # Subtract fee from relative profit if fee is greater than 0
    if fee > 0:
        profit_rel = trade_item['profit_rel'].iloc[0] - fee
        trade_item['profit_rel'] = profit_rel

    # Set Meta-Labels (1: profitable, 0: unprofitable)
    trade_item['bin'] = 1 if profit_rel > 0 else 0

    # Set t1 and t1_price to sell_date and sell_p respectively
    trade_item['t1'] = sell_date
    trade_item['t1_price'] = sell_p

    return trade_item

In [ ]:
def apply_bollinger_bands_v2(trading_daily, data, vertical_barrier_days = 93, fee = 0.006):
    """
    Apply Bollinger Bands trading strategy to the input data (with vertical barrier).

    Parameters:
        trading_daily (dataframe): DataFrame containing daily trading data.
        data (dataframe): DataFrame containing the input data.
        vertical_barrier_days (integer, optional): Number of days for the vertical barrier. Defaults to double the average duration for profitable trades which is 93.
        fee (float, optional): Fee as a percentage. Defaults to 0.06% for a trade with two transactions.

    Returns:
        Dataframe: DataFrame containing the trading results.
    """

    output_list = []

    data = calculate_trade_signals_bb_tbm(data)  # Calculate the trading signals

    trade_item = pd.DataFrame()  # DataFrame to store the current trade item

    for index, item in data.iterrows():
        if item['side'] == -1 and trade_item.empty:  # Sell signal before buy signal, skip it
            continue
        if item['side'] == 1 and trade_item.empty:  # Trade initiation as trade item is still empty
            trade_item = item.to_frame().transpose()
        elif item['side'] == 1 and not trade_item.empty:  # Buy signal but already have a previous buy signal
            if check_if_vertical_barrier_interferes(trade_item, item, vertical_barrier_days):
                # vertical barrier hit, close previous trade and set the current buy signal as new trade
                matching_row = get_vertical_barrier_item(trading_daily, trade_item.index[0], vertical_barrier_days)
                trade_item = add_selling_data(trade_item, matching_row, fee)
                output_list.append(trade_item)

                trade_item = pd.DataFrame()
                trade_item = item.to_frame().transpose()  # Set the second buy signal as the new trade
            else: # buy signal but vertical barrier is not hit, thus second buy signal is ignored
                continue
        elif item['side'] == -1:
            if check_if_vertical_barrier_interferes(trade_item, item, vertical_barrier_days):
                # vertical barrier hit before sell signal, close previous trade with vertical barrier
                matching_row = get_vertical_barrier_item(trading_daily, trade_item.index[0], vertical_barrier_days)
                trade_item = add_selling_data(trade_item, matching_row, fee)
                output_list.append(trade_item)

                # Reset variables
                trade_item = pd.DataFrame()
            else:  # Sell signal occurred before the vertical barrier
                trade_item = add_selling_data(trade_item, item, fee)
                output_list.append(trade_item)

                # Reset variables, prepare for new trade
                trade_item = pd.DataFrame()

    # Check if a trade is open but no data is available, then use last data point available
    if len(trade_item) > 0:
        matching_row = get_vertical_barrier_item(trading_daily, trade_item.index[0], vertical_barrier_days)
        trade_item = add_selling_data(trade_item, matching_row, fee)
        output_list.append(trade_item)

    try:
        output = pd.concat(output_list, axis=0)
    except Exception as e:
        output = pd.DataFrame()

    return output

In [ ]:
# check the average time of profitable trades
df = load_ml_data(path_data_combined, 'v2_bb_and_external_features.csv', vertical_barrier=False)

df_profitable = df[df['bin'] == 1]

df_profitable['duration'] = df_profitable['t1'] - df_profitable['date']

# Calculate the average duration
average_duration = df_profitable['duration'].mean()

# Print the average duration
print(f"Average Duration: {average_duration}")

## TBM-Strategy

In [ ]:
def add_daily_volatility(data, timespan = 25):
    """
    Adds a column 'daily_vol' to the data DataFrame, which represents the daily volatility.

    Parameters:
        data (dataframe): The DataFrame containing the asset data.
        timespan (integer, optional): The number of periods to consider for calculating volatility. Default is 25.

    Returns:
        Dataframe: The updated DataFrame with the 'daily_vol' column added.
    """

    # only add daily volatility if it has not been added before
    if 'daily_vol' not in data:
        # calculate daily volatility
        df_daily_vol = data['close'].pct_change()  # percentage returns
        df_daily_vol = df_daily_vol.ewm(span=timespan).std()  # exponential weighted moving average

        # add column
        data['daily_vol'] = df_daily_vol

        # drop nan values where daily-volatility could not be calculated
        data.dropna(subset=['daily_vol'], inplace=True)
    
    return data

In [ ]:
def add_CUSUM_events(data, threshold):
    """
    Finds CUSUM events in the given data based on a threshold.

    Args:
        data (dataframe): The input data containing 'close' prices.
        threshold (float): The threshold value for detecting CUSUM events.

    Returns:
        Dataframe: Subset of the input data where CUSUM events occur.
    """
    # Initialize variables
    events = []  # CUSUM events
    pos_cum_sum = 0
    neg_cum_sum = 0

    # Calculate log returns
    diff = np.log(data['close']).diff().dropna()

    # Iterate over the log returns
    for i in diff.index[1:]:
        positive = float(pos_cum_sum + diff.loc[i])
        negative = float(neg_cum_sum + diff.loc[i])
        pos_cum_sum = max(0.0, positive)
        neg_cum_sum = min(0.0, negative)

        # check for CUSUM (negative direction)
        if neg_cum_sum < -threshold:
            neg_cum_sum = 0  # Reset
            events.append(i)  # Add CUSUM event

        # check for CUSUM (positive direction)
        elif pos_cum_sum > threshold:
            pos_cum_sum = 0  # Reset
            events.append(i)  # Add CUSUM event

    # Convert events to timestamps
    events_timestamps = pd.DatetimeIndex(events)

    # Return subset of data where CUSUM events occured
    return data.loc[events_timestamps]

In [ ]:
def add_vertical_barrier(data_events, data , num_days = 10):
    """
    Add a vertical barrier to the data_events DataFrame based on the num_days parameter.

    Parameters:
        data_events (dataframe): DataFrame containing event timestamps.
        data (dataframe): DataFrame containing the original data.
        num_days (integer): Number of days to add as a vertical barrier.

    Returns:
        Dataframe: Updated data_events DataFrame with vertical barrier added.
    """
    # Only add vertical barrier if it is not already in our DataFrame
    if 'vertical_barrier' not in data_events:
        events = data_events.index  # Event timestamps
        close = data['close']  # Close prices of all entries in our original data

        # Find next entry in close prices after the event + the num_days
        vb = close.index.searchsorted(events + pd.Timedelta(days=num_days))
        vb = vb[vb < close.shape[0]]  # Check for out of bounds
        vb = pd.Series(close.index[vb], index=events[:vb.shape[0]])  # NaNs at end

        data_events['vertical_barrier'] = vb # Set vertical barrier to dataframe

        # Remove rows with NaNs in the vertical_barrier column
        data_events = data_events.dropna(subset=['vertical_barrier'])
    
    return data_events

In [ ]:
def add_horizontal_barriers(data, data_events, profit_taking, stop_loss):
    """
    Adds horizontal barriers to the data_events DataFrame based on profit-taking and stop-loss limits.

    Args:
        data (dataframe): The main data DataFrame.
        data_events (dataframe): The DataFrame containing the events.
        profit_taking (float): The profit-taking limit.
        stop_loss (float): The stop-loss limit.

    Returns:
        Dataframe: The result DataFrame with horizontal barriers added.
    """

    # Create a copy of the vertical_barrier column
    out = data_events[['vertical_barrier']].copy(deep=True)

    # Set profit-taking and stop-loss limits
    pt = profit_taking * data_events['daily_vol'] if profit_taking > 0 else pd.Series(index=data_events.index)
    sl = -stop_loss * data_events['daily_vol'] if stop_loss > 0 else pd.Series(index=data_events.index)
  
    # Set the vertical barrier to the stop loss or profit taking limit
    for index, vb in data_events['vertical_barrier'].fillna(data['close'].index[-1]).items():
        df0 = data['close'][index:vb]  # Path prices from start of event to vertical barrier
        df0 = (df0 / data['close'][index] - 1) * data_events.at[index, 'side']  # Get path returns

        # Add stop loss and profit taking if possible to out
        out.loc[index, 'sl'] = df0[df0 < sl[index]].index.min()  # Earliest stop loss
        out.loc[index, 'pt'] = df0[df0 > pt[index]].index.min()  # Earliest profit taking
  
    # Get the minimum, which means the first that is reached (stop-loss, profit-taking, vertical)
    data_events.loc[:, 't1'] = out.dropna(how='all').min(axis=1)  # t1 = touch no.1, min of pt and sl

    # Get the prices at the exit t1
    merged_data = pd.merge(data_events['t1'], data['close'], how='left', left_on='t1', right_index=True)
    merged_data['t1_price'] = merged_data['close']
    result_df = pd.concat([data_events, merged_data['t1_price']], axis=1)

    return result_df

In [ ]:
def get_meta_labels(df, fee = 0.006):
    """
    Set meta labels for a given dataframe.

    Parameters:
    df (dataframe): The input dataframe containing the trades and the necessary columns (side, t1_price, close).
    fee (float, optional): The fee to be subtracted from the profit_rel calculation. Default is 0.006 (0.06%).

    Returns:
    Dataframe: The input dataframe with additional columns for profit_rel, and bin.
    """

    # Calculate relative profit
    df['profit_rel'] = df['side'] * ((df['t1_price'] - df['close']) / df['close'])
    
    # Subtract fee from relative profit
    if fee > 0:
        df['profit_rel'] = df['profit_rel'] - fee
        df['profit_abs'] = df['close'] * df['profit_rel']
        
    # Create binary labels based on profit_abs
    # 1: Profitable trade
    # 0: Unprofitable trade
    df['bin'] = df['profit_abs'].apply(lambda x: 1 if x > 0 else 0)

    return df

In [ ]:
def apply_tbm(data, fee, vert_barrier_limit, pt, sl):
    """
    Apply the TBM strategy to the data.

    Args:
        data (dataframe): The input data.
        fee (float): The transaction fee for a trade that contains two transactions (buy and sell).
        vert_barrier_limit (integer): The vertical barrier limit.
        pt (float): The profit taking level.
        sl (float): The stop loss level.

    Returns:
        Dataframe: The modified data with TBM strategy applied.
    """
    # Make a copy of the original data
    data_orig = data.copy(deep=True)

    # Calculate daily volatility
    data = add_daily_volatility(data)

    # Calculate trade signals using Bollinger Bands
    data = calculate_trade_signals_bb_tbm(data)

    # Add CUSUM events
    data = add_CUSUM_events(data, threshold=data['daily_vol'].mean() * 0.1)

    # Add vertical barriers
    data = add_vertical_barrier(data, data_orig, vert_barrier_limit)

    # Add horizontal barriers
    data = add_horizontal_barriers(data_orig, data, pt, sl)

    # Set meta labels
    data = get_meta_labels(data, fee)

    return data

# Meta-Labeling

## Basic Methods

In [ ]:
def label_predictions(predictions, threshold):
    """
    Labels predictions based on a threshold.

    Args:
        predictions (array): Array of predicted probabilities.
        threshold (float): Threshold value for labeling.

    Returns:
        Array: Array of binary labels.
    """
    # Convert predictions to numpy array if not already
    if not isinstance(predictions, np.ndarray):
        predictions = np.array(predictions)

    # Extract positive probabilities from predictions
    positive_probs = predictions[:, 1]

    # Create a binary label array based on the threshold
    predictions_abs = np.where(positive_probs > threshold, 1, 0)

    return predictions_abs

In [ ]:
def label_predictions_fraction(predictions, half_lower_thres, half_upper_thres):
    """
    Labels predictions based on given thresholds for fractional trader.
    
    Args:
        predictions (array): Array of predictions.
        half_lower_thres (float): Lower threshold for labeling as 50% of the investment capital.
        half_upper_thres (float): Upper threshold for labeling as 50% of the investment capital.
        
    Returns:
        Array: Array of labeled predictions.
    """
    # Convert predictions to numpy array if not already
    if not isinstance(predictions, np.ndarray):
        predictions = np.array(predictions)
    
    # Extract positive probabilities
    positive_probs = predictions[:, 1]
    
    # Label predictions based on thresholds
    # label as full investment where probability is higher than lower threshold
    predictions_abs = np.where(positive_probs > half_lower_thres, 1, 0)
    # label as 50% where probability is between lower and upper threshold
    predictions_abs = np.where((positive_probs >= half_lower_thres) & 
                               (positive_probs <= half_upper_thres), 
                               0.5, predictions_abs)
    
    return predictions_abs

In [ ]:
def majority_vote(array_1, array_2, array_3):
    """
    Takes three arrays (predictions) as input and performs a majority vote for each element.
    
    Args:
        array_1 (array): The first array.
        array_2 (array): The second array.
        array_3 (array): The third array.
        
    Returns:
        numpy.ndarray: The result of the majority vote, where each element is either 0 or 1.
    """
    
    # Convert the input arrays to numpy arrays
    array1 = np.array(array_1)
    array2 = np.array(array_2)
    array3 = np.array(array_3)
    
    # Get the length of the arrays
    length = array_1.size
    
    # Create an array of zeros with the same length as the input arrays
    result = np.zeros(length, dtype=np.int64)
    
    # Perform majority vote for each element
    for i in range(length):
        sum_ones = array_1[i] + array_2[i] + array_3[i]
        
        if sum_ones > 1:
            result[i] = 1
    
    return result

In [ ]:
def aggregation_vote(array_1, array_2, array_3):
    """
    Perform aggregation voting on three arrays (predictions).
    
    Args:
        array_1 (array): First input array.
        array_2 (array): Second input array.
        array_3 (array): Third input array.
    
    Returns:
        numpy.ndarray: Resultant array after aggregation voting.
    """
    # Convert arrays to numpy arrays
    array_1 = np.array(array_1)
    array_2 = np.array(array_2)
    array_3 = np.array(array_3)
    
    # Get the length of the arrays
    length = array_1.size
    
    # Initialize result array with zeros
    result = np.zeros(length, dtype=np.int64)
    
    # Perform aggregation voting
    for i in range(length):
        sum_ones = array_1[i] + array_2[i] + array_3[i]
        
        # If sum of ones is equal to 3, set result to 1
        if sum_ones == 3:
            result[i] = 1
    
    return result

In [ ]:
def determine_and_visualize_optimal_threshold(prediction, df_target, filename=None):
    """
    Determine and visualize the optimal threshold based on precision.

    Args:
        prediction (array): The prediction probabilities.
        df_target (dataframe): The target value (either 1 or 0, indicating profitable/unprofitable trade).
        filename (string, optional): The filename to save the visualization. Defaults to None.

    Returns:
        tuple: A tuple containing max_thres, max_val, and precisions.
    """
    thresholds = np.arange(0.5, 1.0, 0.01) # Thresholds to test

    precisions = [] # For visualization store all precision scores
    max_thres = -np.inf # Default value
    max_val = -np.inf # Default value

    for thres in tqdm(thresholds): # Check each threshold and get precision
        prediction_w_thres = label_predictions(prediction, thres)

        prec = precision_score(df_target, prediction_w_thres)
    
        # Check if current precision is highest
        if prec > max_val:
            max_thres = thres
            max_val = prec
        precisions.append(prec)

    print('Max Value at Threshold:', max_thres)

    plt.plot(thresholds, precisions)
    plt.xlabel('Thresholds')
    plt.ylabel('Precision Score')
    plt.title('Threshold Comparison for Precision Score Optimization BB-Strategy')
    plt.xticks(np.arange(0.5, 1.1, 0.1))  # Set x-axis ticks

    # Set the x-axis and y-axis limits to start at 0.0
    plt.xlim(0.5, 1.0)
    plt.ylim(0.0, 1.0)

    # save if filename is given
    if filename is not None:
        plt.savefig(path_default + '/visuals/' + filename, dpi='figure')
    plt.show()
    return max_thres, max_val, precisions

In [ ]:
def calc_overall_profit_for_vis(df, end_date = pd.Period('2023-05')):
    """
    Calculate the overall profit for visualization purposes.

    Args:
        df (dataframe): The input DataFrame containing the data.
        end_date (time period, optional): The end date for the calculation. Defaults to 2023-05 (May 2023).

    Returns:
        tuple: A tuple containing the total returns, months, and returns.
    """

    # Set the column names for grouping
    group_by_t1 = 't1'
    group_by_date = 'date'

    # Convert date columns to datetime
    df['t1'] = pd.to_datetime(df['t1'])
    df['date'] = pd.to_datetime(df['date'])

    # Convert date columns to month periods
    df['month_t1'] = df[group_by_t1].dt.to_period('M')
    df['month_date'] = df[group_by_date].dt.to_period('M')

    # Group the DataFrame by month_t1 and calculate the mean of profit_rel
    grouped_df_t1 = df.groupby('month_t1', as_index=False)['profit_rel'].mean()

    # Rename the column to profit_rel_sum
    grouped_df_t1.rename(columns={'profit_rel': 'profit_rel_sum'}, inplace=True)

    # Determine the start date for the date range
    start_date = grouped_df_t1['month_t1'].min()

    # Generate a date range from start_date to end_date with a monthly frequency
    date_range = pd.period_range(start=start_date, end=end_date, freq='M')

    # Expand the DataFrame by reindexing with the date range and filling missing values with 0
    expanded_df = grouped_df_t1.set_index('month_t1').reindex(date_range, fill_value=0).reset_index()

    # Convert the Period objects to strings
    months = expanded_df['index'].astype(str)

    # Calculate the returns
    returns = expanded_df['profit_rel_sum'].tolist()
    returns_plus_one = np.array(returns) + 1
    total_returns = returns_plus_one.cumprod().tolist()

    return total_returns, months, returns

In [ ]:
def visualize_performance_meta_labeling(df_test, prediction_loaded, show_exclusion = False, bb = True):
    """
    Visualize the performance of Meta-Labeling.

    Args:
        df_test (dataframe): The test data.
        prediction_loaded (array): The prediction values (0 or 1).
        show_exclusion (boolean, optional): Flag to determine whether to show rejected trades. Defaults to False.
        bb (boolean, optional): Flag to determine whether BB-Strategy or TBM-Strategy is used. Defaults to True.
    """
    # Get inclusion and exclusion dataframes
    df, inclusion, exclusion = get_inclusion_exclusion_df(df_test, prediction_loaded, russel=False)
    df_russell, inclusion_russell, exclusion_russell = get_inclusion_exclusion_df(df_test, prediction_loaded, russel=True)
    
    plt.figure(figsize=(10, 6))

    # Calculate necessary values for visualization for all assets
    total_returns_df, months_df, returns_df = calc_overall_profit_for_vis(inclusion)
    total_returns_df_test, months_df_test, returns_df_test = calc_overall_profit_for_vis(df)

    # Calculate necessary values for visualization for Russell assets
    total_returns_df_russell, months_df_russell, returns_df_russell = calc_overall_profit_for_vis(inclusion_russell)
    total_returns_df_test_russell, months_df_test_russell, returns_df_test_russell = calc_overall_profit_for_vis(df_russell)
    
    # Plot all assets total returns
    plt.plot(months_df_test, total_returns_df_test, marker='o', linestyle='-', color='#0E21A0', label='All Assets')
    
    # Plot Russell assets total returns
    plt.plot(months_df_test_russell, total_returns_df_test_russell, marker='o', linestyle='-', color='#900C3F', label='Russell Assets')
    
    if bb:
        # Plot all assets total returns with LGBM 0.9 (best model for BB-Strategy)
        plt.plot(months_df, total_returns_df, marker='o', linestyle='-', color='#9D44C0', label='All Assets w/ LGBM 0.9')
        
        # Plot Russell assets
        plt.plot(months_df_russell, total_returns_df_russell, marker='o', linestyle='-', color='#F94C10', label='Russell Assets w/ LGBM 0.9')
        
        # Set title for BB-Strategy
        plt.title('Total Returns Out-Of-Sample BB-Strategy')
    else:
        # Plot all assets total returns with XGBoost 0.6 (best model for TBM-Strategy)
        plt.plot(months_df, total_returns_df, marker='o', linestyle='-', color='#9D44C0', label='All Assets w/ XGBoost 0.6')
        
        # Plot Russell assets total returns with XGBoost 0.6
        plt.plot(months_df_russell, total_returns_df_russell, marker='o', linestyle='-', color='#F94C10', label='Russell Assets w/ XGBoost 0.6')
        
        # Set title for TBM-Strategy
        plt.title('Total Returns Out-Of-Sample TBM-Strategy')

    if show_exclusion:
        total_returns_df_exc, months_df_exc, returns_df_exc  = calc_overall_profit_for_vis(exclusion)
        total_returns_df_exc_russell, months_df_exc_russell, returns_df_exc_russell  = calc_overall_profit_for_vis(exclusion_russell)

        plt.plot(months_df_exc, total_returns_df_exc, marker='o', linestyle='-', color='#EC53B0', label='All Assets Only Excluded Trades')
        plt.plot(months_df_exc_russell, total_returns_df_exc_russell, marker='o', linestyle='-', color='#F8DE22', label='Russell Assets Only Excluded Trades')

    # Set Configurations for visualization
    plt.xlabel('Month')
    plt.ylabel('Total Return')
    plt.title('Total Returns Out-Of-Sample BB-Strategy')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(path_default + '/visuals/' + 'TBM_XGB_0_6_total_returns', dpi='figure') # Store figure

    plt.show()


## Evaluate Primary and Secondary Models

In [ ]:
def evaluate_model(y_true, y_pred, return_values = False, return_as_df = False, verbose = True):
    """
    Evaluate the performance of a classification model based on precision, recall, f1-score, and accuracy.

    Args:
        y_true (series): True labels.
        y_pred (series): Predicted labels.
        return_values (boolean, optional): Whether to return performance metrics as separate values. Defaults to False.
        return_as_df (boolean, optional): Whether to return performance metrics as a DataFrame. Defaults to False.
        verbose (boolean, optional): Whether to print the performance metrics to console. Defaults to True.

    Returns:
        If return_values is True:
            If return_as_df is True:
                dataframe: Performance metrics as a DataFrame.
            else:
                tuple: Performance metrics as separate values (confusion matrix, precision, accuracy, recall, f1-score).
        else:
            None: Performance metrics plotted as a confusion matrix.
    """

    # Compute the performance metrics
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1_s = f1_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)

    if verbose:
        # Print the metrics to console
        print('')
        print('-' * 40)
        print('Accuracy:', accuracy)
        print('Precision:', precision)
        print('Recall:', recall)
        print('F1-Score:', f1_s)
        print('-' * 40)

    # Generate and return a confusion matrix plot
    cm = confusion_matrix(y_true, y_pred)
    print('CM', cm)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()

    if return_values:
        if return_as_df:
            data = {
                'Accuracy': [accuracy],
                'Precision': [precision],
                'Recall': [recall],
                'F1-Score': [f1_s],
            }
            return pd.DataFrame(data)
        else:
            return cm, precision, accuracy, recall, f1_s
    else:
        return disp.plot()

In [ ]:
def evaluate_primary(df_trading_opportunities, return_values = False, return_as_df = False, verbose = True):
    """
    Prepare necessary series for evaluation. Here, actual and predicted values.

    Args:
        df_trading_opportunities (dataframe): DataFrame all trades as determined by the trading strategy.
        return_values (boolean, optional): If True, returns evaluation metrics as values. Defaults to False.
        return_as_df (boolean, optional): If True, returns evaluation metrics as a DataFrame. Defaults to False.
        verbose (boolean, optional): If True, prints evaluation metrics. Defaults to True.

    Returns:
        Method call: Calling evaluate_model().
    """

    # Create a DataFrame with the 'bin' column from df_trading_opportunities
    primary_forecast = pd.DataFrame(df_trading_opportunities['bin'])

    # Set all predictions as 1 as our primary model only predicts opportunities
    primary_forecast['pred'] = 1

    # Rename 'bin' column to 'actual'
    primary_forecast.columns = ['actual', 'pred']

    # Extract 'actual' and 'pred' columns
    actual = primary_forecast['actual']
    pred = primary_forecast['pred']

    # Call evaluate_model function to evaluate the model's predictions
    return evaluate_model(actual, pred, return_values, return_as_df, verbose)

In [ ]:
def evaluate_primary_and_secondary(bin, secondary_pred, threshold = None, return_values = False, return_as_df = False, verbose = True):
    """
    Evaluate the primary and secondary predictions.

    Args:
        bin: The actual classes of the predictions.
        secondary_pred: The secondary predictions.
        threshold: The threshold for classifying secondary predictions.
        return_values: Whether to return the evaluation values.
        return_as_df: Whether to return the evaluation values as a DataFrame.
        verbose: Whether to print verbose information.

    Returns:
        The evaluation results.

    Raises:
        ValueError: If the secondary predictions contain invalid values.
    """

    # Check input data
    if np.all((secondary_pred == 0) | (secondary_pred == 1) & (threshold is not None)):
        print("Threshold is given even though secondary_pred does not contain confidences.")
        return

    # Apply threshold to the secondary predictions if provided
    if threshold is not None:
        mask = secondary_pred > threshold
        secondary_pred = mask.astype(int)
        
    # Evaluate the model using the primary and secondary predictions
    return evaluate_model(bin, secondary_pred, return_values, return_as_df, verbose)

In [ ]:
def update_trading_df(df_trading, secondary_pred):
    """
    Update the trading dataframe based on a secondary prediction. Method only keeps trades where the secondary model predicted the trade as profitable.

    Args:
        df_trading (dataframe): The original trading dataframe.
        secondary_pred (series): The prediction of the secondary models.

    Returns:
        Dataframe: The updated trading dataframe.
    """
    # Create a copy of the original dataframe
    df_output = df_trading.copy(deep=True)
    
    # Get the trades where the secondary model predicted the trade as profitable
    df_output = df_output[secondary_pred == 1]
    
    return df_output

In [ ]:
def calc_full_evaluation_w_model(model, df, df_X, df_y, save = False):
    """
    Calculate the full evaluation using a model on a given dataframe.

    Args:
        model (machine learning model): The trained model, which is used as the secondary model.
        df (dataframe): The dataframe to test on (here: the out-of-sample data)
        df_X (dataframe): The features of the dataframe df.
        df_y (series): The target variable of the dataframe df.

    Returns:
        Dataframe: A dataframe containing the evaluation for (1) primary strategy (2) Meta-Labeling that includes the secondary prediction (3) Only the trade that were retained by Meta-Labeling.
    """

    # Evaluate primary strategy
    primary_df = evaluate_primary(df, return_values=True, return_as_df=True, verbose=False)

    # Create secondary predictions using the model
    secondary_prediction = model.predict(df_X)

    # Evaluate inclusion of Meta-Labeling, but keep the trades that were predicted as unprofitable
    secondary_df = evaluate_model(df_y, secondary_prediction, return_values=True, return_as_df=True, verbose=False)

    # Create dataframe that only contains trades which were validated by Meta-Labeling
    updated_tradings = update_trading_df(df, secondary_prediction)

    # Evaluate performance of the trades that were validated by Meta-Labeling
    updated_df = evaluate_primary(updated_tradings, return_values=True, return_as_df=True, verbose=False)

    # Create dataframe
    primary_df['model'] = 'primary'
    secondary_df['model'] = 'secondary'
    updated_df['model'] = 'updated'
    performance = pd.concat([primary_df, secondary_df, updated_df], ignore_index=True)

    if save:
        performance_json = performance.to_json(orient='records')

        with open(path_default + '/overall_performance.json', 'w') as file:
            file.write(performance_json)

    return performance



def calc_full_evaluation_w_prediction(prediction, df, df_X, df_y, save = False):
    """
    Calculate the full evaluation using the predictions of a secondary model on a given dataframe.

    Args:
        prediction (array): The predictions of the secondary model on the out-of-sample data.
        df (dataframe): The dataframe to test on (here: the out-of-sample data)
        df_X (dataframe): The features of the dataframe df.
        df_y (series): The target variable of the dataframe df.

    Returns:
        dataframe: A dataframe containing the evaluation for (1) primary strategy (2) Meta-Labeling that includes the secondary prediction (3) Only the trade that were retained by Meta-Labeling.
    """
    
    # Evaluate primary strategy
    primary_df = evaluate_primary(df, return_values=True, return_as_df=True, verbose=False)
    
    # Evaluate inclusion of Meta-Labeling, but keep the trades that were predicted as unprofitable
    secondary_df = evaluate_model(df_y, prediction, return_values=True, return_as_df=True, verbose=False)

    # Create dataframe that only contains trades which were validated by Meta-Labeling
    updated_tradings = update_trading_df(df, prediction)

    # Evaluate performance of the trades that were validated by Meta-Labeling
    updated_df = evaluate_primary(updated_tradings, return_values=True, return_as_df=True, verbose=False) # updated scores

    # Create dataframe
    primary_df['model'] = 'primary'
    secondary_df['model'] = 'secondary'
    updated_df['model'] = 'updated'
    performance = pd.concat([primary_df, secondary_df, updated_df], ignore_index=True)
    
    if save:
        performance_json = performance.to_json(orient='records')
        with open(path_default + '/overall_performance.json', 'w') as file:
            file.write(performance_json)
    
    return performance

## Financial Performance Evaluation

In [ ]:
# Helper methods to assess performance metrics based on individual assets

def order_trades_by_date(df):
    """
    Orders trades within a DataFrame by date for each group of trades with the same 'cusip' value.

    Args:
        df (dataframe): All trades as determined by the primary strategy.

    Returns:
        Dataframe: The dataframe with the trades ordered by date for each group of trades.
    """
    # Sort each group by the 'date' column within the DataFrame
    df = df.groupby('cusip', group_keys = False).apply(lambda x: x.sort_values('date')).reset_index(drop = True)
    return df

def set_cusip_date_index(df):
    """
    Set the 'date' column as a datetime and set the index of the dataframe based on 'cusip' and 'date' columns.

    Args:
        df (dataframe): All trades as determined by the primary strategy.

    Returns:
        Dataframe: The Dataframe with the 'date' column as a datetime and the index set based on 'cusip' and 'date' columns.
    """

    df['date'] = pd.to_datetime(df['date'])
    df_index = df.set_index(['cusip', 'date']).sort_values(['cusip', 'date'])

    return df_index

In [ ]:
def calc_avg_maximum_loss(df):
    """
    Calculate the maximum loss for each unique asset in the given dataframe.

    Args:
        df (dataframe): The input dataframe that contains trades, whereby the cusip and date need to be the index.

    Returns:
        Tuple: A tuple containing the maximum loss and a list of all the individual maximum losses.

    """
    # Get all unique assets
    tokens = df.index.get_level_values(0).unique()

    # List to store the individual maximum losses
    minimums = []

    # Loop through each unique asset and calculate the maximum loss
    for token in tqdm(tokens, desc='Maximum Loss Calculation'):

        # Get sub-df which only contains assets for the given unique cusip identifier
        df_token = df.loc[token]

        # Maximum loss = minimum relative profit
        min_loss = df_token['profit_rel'].min()

        # Only add if it is a loss as the minimum relative profit can be positive, too
        if min_loss < 0:
            minimums.append(min_loss)

    # Calculate the average
    avg_loss = np.mean(minimums)

    return avg_loss, minimums

In [ ]:
def calc_avg_maximum_drawdown(df):
    """
    Calculate the maximum drawdown for all trades in given dataframe.

    Args:
        df (dataframe): The input dataframe that contains trades, whereby the cusip and date need to be the index.

    Returns:
        Tuple: A tuple containing the maximum drawdown value and a list of drawdown values for each token.
    """
    # Get all unique cusip values
    tokens = df.index.get_level_values(0).unique()
    
    drawdowns = []
    
    for token in tqdm(tokens, desc='Maximum Drawdown Calculation'):
        df_token = df.loc[token]
        
        # Calculate cumulative return
        df_token['cumulative_return'] = df_token['profit_rel'].cumsum() + 1
        
        # Calculate peak value
        df_token['peak_value'] = df_token['cumulative_return'].cummax()
        
        # Calculate drawdown
        df_token['drawdown'] = (df_token['peak_value'] - df_token['cumulative_return']) / df_token['peak_value']
        
        # Calculate maximum drawdown and append to drawdowns list
        maximum_drawdown = df_token['drawdown'].max()
        drawdowns.append(maximum_drawdown)       

    # Calculate average maximum drawdown
    avg_maximum_drawdown = np.mean(drawdowns)
    
    return avg_maximum_drawdown, drawdowns

In [ ]:
def calc_profits_trading_strat(df):
    """
    Calculate the average profit per trade for the trading strategy.

    Args:
        df (dataframe): The trades.

    Returns:
        Float: The average profit per trade.
    """
    return df['profit_rel'].mean()

In [ ]:
def calc_profit_factor(df):
    """
    Calculates the profit factor for given trades.

    Args:
        df (dataframe): The trades.

    Returns:
        Float: The calculated profit factor.
    """
    # Filter the DataFrame to get profitable and unprofitable trades.
    df_pos = df[df['bin'] == 1]
    df_neg = df[df['bin'] == 0]

    # Calculate the sum of the relative profits for each group
    pos_sum = df_pos['profit_rel'].sum()
    neg_sum = df_neg['profit_rel'].sum()

    profit_factor = pos_sum / abs(neg_sum)

    return profit_factor

In [ ]:
def calc_hit_ratio(df):
    """
    Calculate the hit ratio.

    Args:
        df (dataframe): The trades.
    Returns:
        Float: The hit ratio as a percentage.
    """
    number_of_trades = len(df)  
    winning_trades = df[df['bin'] == 1] 
    return len(winning_trades) / number_of_trades

In [ ]:
def calc_overall_profit(df):
    """
    Calculate the overall profit that can be used to calculate the annualized profits.

    Args:
        df (dataframe): The trades.

    Returns:
        Tuple: A tuple containing the overall profit and the number of months where trades occurred.
    """

    column_to_group_by = 't1' # Group by the selling date (t1 = selling date)

    # Convert column to datetime
    df[column_to_group_by] = pd.to_datetime(df[column_to_group_by])

    # Group by month and calculate average profit_rel per month
    df['month'] = df[column_to_group_by].dt.to_period('M')
    grouped_df = df.groupby('month', as_index=False)['profit_rel'].mean()
    grouped_df.rename(columns={'profit_rel': 'profit_rel_sum'}, inplace=True)

    # Calculate returns
    returns = grouped_df['profit_rel_sum'].tolist()
    returns_plus_one = np.array(returns) + 1
    total_returns = returns_plus_one.cumprod().tolist()

    # Overall profit at the end of the available months - 1 (initial investment)
    overall_profit = total_returns[-1] - 1
    num_months = len(grouped_df)

    return overall_profit, num_months

In [ ]:
def full_financial_performance_eval(df, prediction, fraction=False, russel=True):
    """
    Conduct a full financial performance for given trades.

    Args:
        df (dataframe): The trades.
        prediction (array): The prediction of the secondary model.
        fraction (boolean, optional): If true, fractional investments are given in the prediction array. Defaults to False.
        russel (boolean, optional): Indicates whether only Russell 3000 assets should be considered. Defaults to True.
    """
    # Load buy and hold performance data
    df_bnh = load_ml_data(path_data_buy_hold, 'v1_BB_buy_n_hold_with_fees.csv', vertical_barrier=False)

    print('Russel Activated?:', russel)
    if fraction:
        mask_remove = prediction == 0
        mask_half = prediction == 0.5
        mask_keep = prediction == 1
        
        updated_trades_keep = df[mask_keep] # Trades to keep
        updated_trades_half = df[mask_half] # Trades to half the investment capital
        updated_trades_half['profit_rel'] /= 2

        # Combine Trades that are conducted
        updated_trades = updated_trades_keep.append(updated_trades_half, ignore_index = True)
        
        original_trades = df.copy(deep=True)

        # Trades that were rejected
        deleted_trades = df[mask_remove]
    else:
        mask_remove = prediction == 0
        mask_keep = prediction == 1
        updated_trades = df[mask_keep] # Validated trades
        original_trades = df.copy(deep=True) # All trades
        deleted_trades = df[mask_remove] # Rejected trades
        
    # check if you want to only take the russel assets
    if russel:
        # Read CUSIPs that were part of the Russell 3000
        russel_2020 = pd.read_csv(path_data_combined + '/cusips_from_2020.csv')
        russel_cusips = russel_2020['0'].tolist()
        
        updated_trades = updated_trades[updated_trades['cusip'].isin(russel_cusips)]
        original_trades = original_trades[original_trades['cusip'].isin(russel_cusips)]
        deleted_trades = deleted_trades[deleted_trades['cusip'].isin(russel_cusips)]      
        
        df_bnh = df_bnh[df_bnh['cusip'].isin(russel_cusips)]

    # Filter buy and hold by trades that were validated by Meta-Labeling
    remaining_cusips = updated_trades['cusip'].unique()
    print('#Remainig:', len(remaining_cusips))
    df_bnh = df_bnh[df_bnh['cusip'].isin(remaining_cusips)]
        
    print('Number of Trades Original', len(original_trades))
    print('Number of Trades Updated', len(updated_trades))
    
    profit_overall_or, no_months_orig = calc_overall_profit(original_trades)
    profit_overall_up, no_month_upd = calc_overall_profit(updated_trades)
    # Annualized the profits by diving by the number of months and multiplying with 12
    print('Overall Profits Original per year:', (profit_overall_or / no_months_orig) * 12)
    print('Overall Profits Updated per year:', (profit_overall_up / no_month_upd) * 12)
    
    # Order trades by date and set index of dataframes as date and cusip
    updated_trades = order_trades_by_date(updated_trades)
    updated_trades = set_cusip_date_index(updated_trades)
    original_trades = order_trades_by_date(original_trades)
    original_trades = set_cusip_date_index(original_trades)

    # Maximum Loss
    avg_max_loss_original, _ = calc_avg_maximum_loss(original_trades)
    avg_max_loss_updated, _ = calc_avg_maximum_loss(updated_trades)
    print('AVG Max. Loss Original:', avg_max_loss_original)
    print('AVG Max. Loss Updated:', avg_max_loss_updated)
    
    # Maximum Drawdown
    avg_max_drawdown_original, _ = calc_avg_maximum_drawdown(original_trades)
    avg_max_drawdown_updated, _ = calc_avg_maximum_drawdown(updated_trades)
    print('Average Maximum Drawdown Original:', avg_max_drawdown_original)
    print('Average Maximum Drawdown Updated:', avg_max_drawdown_updated)
    
    # Profit Factor
    profit_factor_original = calc_profit_factor(original_trades)
    profit_factor_updated = calc_profit_factor(updated_trades)
    print('Profit Factor Original:', profit_factor_original)
    print('Profit Factor Updated:', profit_factor_updated)
    
    # Hit Ratio
    hit_ratio_original = calc_hit_ratio(original_trades)
    hit_ratio_updated = calc_hit_ratio(updated_trades)
    print('Hit Ratio Original:', hit_ratio_original)
    print('Hit Ratio Updated:', hit_ratio_updated)
    
    # Average Profit per Trade
    profit_avg_or = calc_profits_trading_strat(original_trades)
    profit_avg_up = calc_profits_trading_strat(updated_trades)
    print('Average Profit per Trade Original:', profit_avg_or)
    print('Average Profit per Trade Updated:', profit_avg_up)
    
    # Buy and Hold Performance annualized
    print('Profits Buy and Hold:', df_bnh['profit_rel'].mean() / no_months_orig * 12)

In [ ]:
def get_inclusion_exclusion_df(df, prediction, russel=False):
    """
    Get dataframe of validated and rejected trades.

    Args:
        df (dataframe): The trades.
        prediction (array): The prediction of the secondary model.
        russel (boolean, optional): Indicates whether only Russell 3000 assets should be considered. Defaults to True.
    Returns:
        Tuple: A tuple containing all trades, validated trades, and rejected trades
    """
    df_all = df.copy(deep=True)
    df_excluded = df.copy(deep=True)
    df_included = df.copy(deep=True)

    df_included = df_included[prediction == 1] # Profitable trades
    df_excluded = df_excluded[prediction == 0] # Unprofitable trades

    if russel: # Filter by CUSIPs that were part of the Russell 3000
        russel_2020 = pd.read_csv(path_data_combined + '/cusips_from_2020.csv')
        russel_cusips = russel_2020['0'].tolist()
        
        df_included = df_included[df_included['cusip'].isin(russel_cusips)]
        df_excluded = df_excluded[df_excluded['cusip'].isin(russel_cusips)]
        df_all = df_all[df_all['cusip'].isin(russel_cusips)]
    return df_all, df_included, df_excluded


# ML Methods

## General

In [ ]:
def save_model_sklearn(model, path, model_name):
    """
    Saves models from sklearn to filesystem.

    Args:
        model (sklearn model): Sklearn model.
        path (string): The path to save the model to.
        model_name (string): The name of the model.
    """
    joblib.dump(model, path + '/' + model_name + '.joblib')

def load_model_sklearn(path, model_name):
    """
    Loads sklearn model from filesystem.

    Args:
        path (string): The path where the model is stored.
        model_name (string): The name of the model.
    Returns:
        Model: The loaded model.
    """
    model = joblib.load(path + '/' + model_name + '.joblib')
    return model

def store_prediction_csv(prediction, filename, path=path_default):
    """
    Saves prediction of a model to filesystem.

    Args:
        prediction (array): The predicted values of the secondary model.
        filename (string): The name of the file.
        path (string, optional): The path to save the file. Defaults to the default file system path as configured in the notebook.
    """
    df = pd.DataFrame(prediction)
    df.to_csv(path + '/data_v5/predictions/' + filename, index=False, header=False)

def load_prediction_csv(filename, proba = False, path = path_default):
    """
    Loads prediction of a model from filesystem.

    Args:
        filename (string): The name of the file.
        proba (boolean, optional): 
        path (string, optional): The path to save the file. Defaults to the default file system path as configured in the notebook.
    Returns:
        Either the absolute values (0 or 1), or the predicted probabilities (depending on the value of proba).
    """
    df = pd.read_csv(path + '/data_v5/predictions/' + filename, header=None)
    if proba:
        return df.values
    else:
        return np.ravel(df.values) 

In [ ]:
def train_model_k_fold(model, config, X, y, threshold = 0.5):
    """
    Train a model using k-fold cross validation.

    Args:
        model (model): The model that should be trained.
        config (dict): The configuration for the model.
        X (dataframe): The features.
        y (series): The targets.
        threshold (float, optional): The threshold to label a trade as profitable. Defaults to 0.5.
    """
    k_fold = KFold(n_splits = 3, random_state = 42, shuffle = True)
    pre, acc, rec, f1_s, mcc_s = [], [], [], [], [] # Lists to store the evaluation metrics
    for fold, (train_idx, test_idx) in tqdm(enumerate(k_fold.split(X, y)), total=k_fold.get_n_splits(), desc='k-fold:          '):
        model.set_params(**config) 
        model.fit(X.iloc[train_idx], y.iloc[train_idx])
        
        # Predict Probabilities
        y_pred_proba = model.predict_proba(X.iloc[test_idx])

        # Get absolute predictions using the threshold
        y_pred = label_predictions(y_pred_proba, threshold)

        # Evaluate model
        _, fold_precision, fold_accuracy, fold_recall, fold_f1, fold_mcc = evaluate_model(y.iloc[test_idx], y_pred, return_values=True, verbose=False) # evaluate
        
        # Store metrics
        pre.append(fold_precision)
        acc.append(fold_accuracy)
        rec.append(fold_recall)
        f1_s.append(fold_f1)
        mcc_s.append(fold_mcc)
    
    # Average metrics
    precision = sum(pre) / len(pre)
    accuracy = sum(acc) / len(acc)
    recalll = sum(rec) / len(rec)
    f1 = sum(f1_s) / len(f1_s)
    mcc = sum(mcc_s) / len(mcc_s)
    
    # Print performance
    print('Precision', precision)
    print('Accuracy', accuracy)
    print('Recall', recalll)
    print('F1', f1)
    print('MCC', mcc)

In [ ]:
def train_model(model, config, X, y):
    """
    Train a model by fitting the features to the targets.

    Args:
        model (model): The model that should be trained.
        config (dict): The configuration for the model.
        X (dataframe): The features.
        y (series): The targets.
    """
    model.set_params(**config)
    model.fit(X.values, y.values)
    return model

In [ ]:
def train_model_k_fold_feature_imp(model, config, X, y):
    """
    Train a model and get feature importance as average across all folds.

    Args:
        model (model): The model that should be trained.
        config (dict): The configuration for the model.
        X (dataframe): The features.
        y (series): The targets.
    Returns:
        Dataframe: The feature importances averaged across all folds.
    """
    feature_imp = pd.DataFrame()
    k_fold = KFold(n_splits = 3, random_state = 42, shuffle = True)
    for fold, (train_idx, test_idx) in tqdm(enumerate(k_fold.split(X, y)), total = k_fold.get_n_splits(), desc = 'k-fold: '):
        model.set_params(**config) # set given params
        model.fit(X.iloc[train_idx], y.iloc[train_idx]) # fit on training data
        
        # Retrieve feature importances
        fold_feature_imp = pd.DataFrame(sorted(zip(model.feature_importances_, X.columns)), columns = ['Value','Feature'])
        feature_imp = pd.concat([feature_imp, fold_feature_imp], axis=0)

    # Group by 'Feature' and calculate the mean feature importance across all folds
    feature_imp = feature_imp.groupby('Feature')['Value'].mean().reset_index()
    feature_imp = feature_imp.sort_values(by = 'Value', ascending = False).reset_index(drop = True)

    return feature_imp

## ANN

In [ ]:
# Neural Network Configuration for BB-Strategy
class CustomMLP_BB(nn.Module):
    def __init__(self, input_shape):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_shape, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

# Neural Network Configuration for TBM-Strategy
class CustomMLP_TBM(nn.Module):
    def __init__(self, input_shape):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_shape, 128),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
# Create Dataset for the given data
class CustomStockDataSet(torch.utils.data.Dataset):
    def __init__(self, X, y):
        if not torch.is_tensor(X) and not torch.is_tensor(y):
            self.X = torch.from_numpy(X.values)
            self.y = torch.from_numpy(y.values)

            self.X = self.X.to(torch.float32)
            self.y = self.y.to(torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

In [ ]:
def evaluate_model_nn(y_pred, y_test):
    """
    Evaluate the performance of a neural network.

    Args:
        y_pred (tensor): The predictions based on the neural network.
        y_test (tensor): The actual values based of the trades (profitable or unprofitable).
    Returns:
        Tuple: A tuple containing the accuracy, precision, recall and the f1-score.
    """
    y_pred_rounded = torch.round(y_pred)
    y_pred_list = [item for sublist in y_pred_rounded.tolist() for item in sublist]
    y_test_list = [item for sublist in y_test.tolist() for item in sublist]

    acc = accuracy_score(y_test_list, y_pred_list)
    precision = precision_score(y_test_list, y_pred_list)
    recall = recall_score(y_test_list, y_pred_list)
    f1 = f1_score(y_test_list, y_pred_list)
    return acc, precision, recall, f1

In [ ]:
def save_model_performance(path, df, fold):
    """
    Saving the performance of a model to the filesystem.

    Args:
        path (string): The path to save the performance.
        df (dataframe): The dataframe that contains the performance and which should be saved.
        fold (int): The fold number of the given performance.
    """
    output_dir = path + '/' + 'fold_' + str(fold)
    try:
        os.mkdir(output_dir)
    except:
        pass
    df.to_csv(output_dir +  '/' +  'performance.csv', index=False, encoding='utf-8')


In [ ]:
def get_structure_dict(nn):
    """
    Getting the dictionary with the structure of the neural network (PyTorch).

    Args:
        nn (model): The neural network.
    Returns:
        Dictionary: The structure of the neural network.
    """
    structure_dict = {}
    for name, module in nn.named_modules():
        if name == '':
            structure_dict[name] = {'input_shape': [None] + list(nn.layers[0].weight.size())[1:]}
        else:
            structure_dict[name] = str(module)
    return structure_dict


In [ ]:
def save_config(path, config):
    """
    Saving the configuration of the neural network to the filesystem.
    
    Args:
        path (string): The path to save the configuration.
        config (dict): The configuration of the neural network.
    """
    with open(path + '/configuration.json', 'w') as f:
        json.dump(config, f)

In [ ]:
def load_model(model, path):
    """
    Loading a neural network from the filesystem.

    Args:
        model (model): The neural network model.
        path (string): The path to load the model from.
    Returns:
        model: The loaded neural network.
    """
    model.load_state_dict(torch.load(path))
    model.eval() # Set to eval mode
    return model


def save_model(model, epoch, path):
    """
    Saving a neural network to the filesystem.

    Args:
        model (model): The neural network model.
        epoch (int): The epoch number of the neural network.
        path (string): The path to save the model to.
    """
    torch.save(model.state_dict(), path + '/epoch_' + str(epoch))

In [ ]:
def save_html_loss_plot_to_file(path, losses_train, losses_val):
    """
    Saving the loss plots to an HTML file.
    
    Args:
        path (string): The path to save the plot to.
        losses_train (list): The training losses.
        losses_val (list): The validation losses.
    """

    # Generate x-axis values (epochs)
    epochs = list(range(1, len(losses_train) + 1))

    losses_tr_cpu = [tensor.cpu().detach().numpy() for tensor in losses_train]
    losses_vl_cpu = [tensor.cpu().detach().numpy() for tensor in losses_val]

    # Create the interactive line plot
    fig = go.Figure()

    # Add train and validation loss traces
    fig.add_trace(go.Scatter(x=epochs, y=losses_tr_cpu, mode='lines', name='Train Loss'))
    fig.add_trace(go.Scatter(x=epochs, y=losses_vl_cpu, mode='lines', name='Validation Loss'))

    # Add labels and title
    fig.update_layout(title='Train and Validation Losses', xaxis_title='Epochs', yaxis_title='Losses')

    # Save the plot as an HTML file
    fig.write_html(path + '/losses.html')

In [ ]:
def save_html_performance_plot_to_file(path, pre_t, pre_v, rec_t, rec_v, acc_t, acc_v, f1_t, f1_v):
    """
    Saving the plot of the performance metrics to an HTML file.
    
    Args:
        path (string): The path to save the plot to.
        pre_t (list): The precision performance metrics on training.
        pre_v (list): The precision performance metrics on validation.
        rec_t (list): The recall performance metrics on training.
        rec_v (list): The recall performance metrics on validation.
        acc_t (list): The accuracy performance metrics on training.
        acc_v (list): The accuracy performance metrics on validation.
        f1_t (list): The f1-score performance metrics on training.
        f1_v (list): The f1-score performance metrics on validation.
    """

    # Generate x-axis values (epochs)
    epochs = list(range(1, len(pre_t) + 1))

    # Create the interactive line plot
    fig = go.Figure()

    # Add train and validation loss traces
    fig.add_trace(go.Scatter(x=epochs, y=pre_t, mode='lines', name='PRE Train'))
    fig.add_trace(go.Scatter(x=epochs, y=pre_v, mode='lines', name='PRE Valid'))
    fig.add_trace(go.Scatter(x=epochs, y=rec_t, mode='lines', name='REC Train'))
    fig.add_trace(go.Scatter(x=epochs, y=rec_v, mode='lines', name='REC Valid'))
    fig.add_trace(go.Scatter(x=epochs, y=acc_t, mode='lines', name='ACC Train'))
    fig.add_trace(go.Scatter(x=epochs, y=acc_v, mode='lines', name='ACC Valid'))
    fig.add_trace(go.Scatter(x=epochs, y=f1_t, mode='lines', name='F1 Train'))
    fig.add_trace(go.Scatter(x=epochs, y=f1_v, mode='lines', name='F1 Valid'))

    # Add labels and title
    fig.update_layout(title='Train and Validation Performances', xaxis_title='Epochs', yaxis_title='Percentages')

    # Save the plot as an HTML file
    fig.write_html(path + '/performance.html')

In [ ]:
def train_model(model, train_dl, val_dl , epochs, optimizer, loss_func, fold, execution_path):
    """
    Training loop for neural network (PyTorch).

    Args:
        model (model): The neural network model.
        train_dl (dataloader): The dataloader containing the training data.
        val_dl (dataloader): The dataloader containing the validation data.
        epochs (int): The maximum number of epochs.
        optimizer (optimizer): The optimizer.
        loss_func (loss_func): The loss function.
        fold (int): The fold number.
        execution_path (string): The path to save the model to.
    Returns:
        Tuple: Training and validation losses.
    """
    # Get GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    wandb.watch(model, loss_func, log='all', log_freq = 100)

    print('Device: ', torch.cuda.get_device_name(0))
    print("_"*100)
    print("epoch  | tr-loss  | tr-acc   | tr-f1    | te-loss  | te-acc    | te-f1    ")

    # Move model to GPU
    model = model.to(device = device)
    performance_list = []
    
    # Loss
    all_losses_train = []
    all_losses_val = []
    
    # Precision
    all_precision_train = []
    all_precision_val = []
    
    # Accuracy
    all_accuracy_train = []
    all_accuracy_val = []
    
    # Recall
    all_recall_train = []
    all_recall_val = []
    
    # F1-score
    all_f1_train = []
    all_f1_val = []
    
    for epoch in range(epochs):
        model.train()
        total_loss, total_acc, total_precision, total_recall, total_f1 = 0.0, 0.0, 0.0, 0.0, 0.0 # Initialize values
        
        for xb, yb in train_dl:
            xb, yb  = xb.to(device = device), yb.to(device = device)
            
            pred = model(xb)
            
            yb = yb.unsqueeze(1)
            yb = yb.float()
            
            loss = loss_func(pred, yb)
            
            acc, precision, recall, f1 = evaluate_model_nn(pred, yb)
            
            total_loss += loss
            total_acc += acc
            total_precision += precision
            total_recall += recall
            total_f1 += f1
            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        total_loss /= len(train_dl)
        total_acc /= len(train_dl)
        total_precision /= len(train_dl)
        total_recall /= len(train_dl)
        total_f1 /= len(train_dl)
        

        total_loss_val, total_acc_val, total_precision_val, total_recall_val, total_f1_val = 0.0, 0.0, 0.0, 0.0, 0.0
        model.eval()
        
        with torch.no_grad():
            for xb_val, yb_val in val_dl:
                xb_val, yb_val = xb_val.to(device = device), yb_val.to(device = device)
                
                pred_val = model(xb_val)
                
                yb_val = yb_val.unsqueeze(1)
                yb_val = yb_val.float()
                
                if torch.all(pred_val == 0):
                    pass
                elif torch.all(pred_val == 1):
                    pass

                loss_val = loss_func(pred_val, yb_val)
                
                acc_val, precision_val, recall_val, f1_val = evaluate_model_nn(pred_val, yb_val)
                
                total_loss_val += loss_val
                total_acc_val += acc_val
                total_precision_val += precision_val
                total_recall_val += recall_val
                total_f1_val += f1_val
                
        total_loss_val /= len(val_dl)
        total_acc_val /= len(val_dl)
        total_precision_val /= len(val_dl)
        total_recall_val /= len(val_dl)
        total_f1_val /= len(val_dl)
        
        model = model.to(device) 

        performance = {
            'epoch': epoch + 1,
            'train_loss': total_loss.item(),
            'val_loss': total_loss_val.item(),
            'train_acc': total_acc.item(),
            'val_acc': total_acc_val.item(),
            'train_f1': total_f1.item(),
            'val_f1': total_f1_val.item(),
            'train_precision': total_precision.item(),
            'val_precision': total_precision_val.item(),
            'train_recall': total_recall.item(),
            'val_recall': total_recall_val.item(),
        }

        performance_list.append(performance)
        
        all_losses_train.append(total_loss)
        all_losses_val.append(total_loss_val)
        
        all_precision_train.append(total_precision)
        all_precision_val.append(total_precision_val)
        
        all_recall_train.append(total_recall)
        all_recall_val.append(total_recall_val)
        
        all_accuracy_train.append(total_acc)
        all_accuracy_val.append(total_acc_val)
        
        all_f1_train.append(total_f1)
        all_f1_val.append(total_f1_val)
        
        # Log performance to wandb
        wandb.log({
            f"epoch" : epoch, 
            f"loss_train" : total_loss, 
            f"loss_val" : total_loss_val, 
            f"precision_train" : total_precision, 
            f"precision_val" : total_precision_val, 
            f"recall_train" : total_recall, 
            f"recall_val" : total_recall_val, 
            f"accuracy_train" : total_acc, 
            f"accuracy_val" : total_acc_val, 
            f"f1_train" : total_f1, 
            f"f1_val" : total_f1_val, 
        })
        
        # Buffering
        if (epoch % 50 == 0) & (epoch != 0):
            save_model(model, epoch, execution_path)
        
        print("_"*100)
        print(f"  {epoch + 1}    |  {total_loss.item():.4f}  |  {(total_acc.item()*100):.4f} | {(total_f1.item()*100):.4f}  | {total_loss_val.item():.4f}   |  {(total_acc_val.item()*100):.4f}  |{(total_f1_val.item()*100):.4f}   ")
        print()

    df_performance = pd.DataFrame(performance_list)
    save_model_performance(execution_path, df_performance, fold)
    save_html_loss_plot_to_file(execution_path, all_losses_train, all_losses_val)
    save_html_performance_plot_to_file(execution_path, all_precision_train, all_precision_val, all_recall_train, all_recall_val, all_accuracy_train, all_accuracy_val, all_f1_train, all_f1_val )
    save_model(model, epoch, execution_path)
    return all_losses_train, all_losses_val

# Visualization Trading Strategies

## BB-Strategy

In [ ]:
df_vis = read_data_csv()

# AAPL = 037833100
df_vis_sub = df_vis[df_vis['cusip'] == '037833100']

# Get trading signals
df_vis_trd = calculate_bb(df_vis_sub)

df_vis_trd = df_vis_trd.set_index(df_vis_trd['date'], drop=True)

df = df_vis_trd.copy(deep=True)

df_sgnls = calculate_trade_signals_bb_tbm(df_vis_trd)

# Set boundaries for visualization purposes
df = df.loc['2018-01-01':'2018-12-31']
df_sgnls = df_sgnls.loc['2018-01-01':'2018-12-31']

In [ ]:
# Visualize
fig, ax = plt.subplots(figsize=(10, 6))

x_axis = df.index

# Plotting the Bollinger Bands
ax.plot(df.index, df['close'], label='Closing Prices', color='blue')
ax.plot(df.index, df['upper_bb'], label='Upper Band', color='turquoise')
ax.plot(df.index, df['lower_bb'], label='Lower Band', color='orange')
ax.plot(df.index, df['sma_20'], label='Mean', color='black')

buy_signals = df_sgnls[df_sgnls['side'] == 1]
sell_signals = df_sgnls[df_sgnls['side'] == -1]
ax.scatter(buy_signals.index, buy_signals['close'], color='green', lw=0.001, label='Buy Signal', marker='^', s=100, zorder=10)
ax.scatter(sell_signals.index, sell_signals['close'], color='red', lw=0.001, label='Sell Signal', marker='v', s=100, zorder=10)
ax.fill_between(x_axis, df['upper_bb'], df['lower_bb'], color='grey', alpha=0.15)

# Adding labels and title
ax.set_xlabel('Date')
ax.set_ylabel('Price in US-$')

# Adding legend and grid
ax.legend()
ax.grid(True)

# Rotating x-axis labels for better readability
plt.xticks(rotation=45)

fig.tight_layout(pad=1.5)
plt.savefig(path_default + '/visuals/bb.svg', format='svg')
    
# Display the plot
plt.show()

## TBM-Strategy

In [ ]:
df_all = read_data_csv()
df_all = df_all.set_index('date', drop=True)

# AAPL = 037833100
df_aapl = df_all[df_all['cusip'] == '037833100']

# Set boundaries
df = df_aapl.loc['2021-01-01':'2021-04-01']

In [ ]:
# Specify the date and corresponding price for the first box
box1_date = '2021-01-12'  # Change this to your desired date
box1_price = df.loc[box1_date, 'close']
box1_height = 10  # Change this to your desired height

# Specify the date and corresponding price for the second box
box2_date = '2021-02-01'  # Change this to your desired date
box2_price = df.loc[box2_date, 'close']
box2_height = 10  # Change this to your desired height

# Specify the date and corresponding price for the third box
box3_date = '2021-03-01'  # Change this to your desired date
box3_price = df.loc[box3_date, 'close']
box3_height = 10  # Change this to your desired height

# Create the plot
fig, ax = plt.subplots(figsize=(10, 6))

x_axis = df.index

ax.plot(df.index, df['close'], label='Closing Price', color='blue')

# Adding labels and title
ax.set_xlabel('Date')
ax.set_ylabel('Price in US-$')

# Adding legend and grid
ax.legend()
ax.grid(True)

# Set x-axis limits to ensure it goes until '2023-04-01'
ax.set_xlim(df.index[0], pd.Timestamp('2021-04-01'))

# Rotating x-axis labels for better readability
plt.xticks(rotation=45)

# Add the first box
box_width1 = pd.Timedelta(days=15)  # Box width of 15 days
box_start1 = pd.to_datetime(box1_date)

# Add the second box
box_width2 = pd.Timedelta(days=15)  # Box width of 10 days
box_start2 = pd.to_datetime(box2_date)

# Add the third box
box_width3 = pd.Timedelta(days=15)  # Box width of 15 days
box_start3 = pd.to_datetime(box3_date)

# Calculate the maximum and minimum prices for the vertical lines
max_price1 = df['close'].max()
min_price1 = df['close'].min()

max_price2 = df['close'].max()
min_price2 = df['close'].min()

max_price3 = df['close'].max()
min_price3 = df['close'].min()

# Calculate the maximum height for the vertical lines
max_height1 = min(box1_height / 2, max_price1 - box1_price)
min_height1 = min(box1_height / 2, box1_price - min_price1)

max_height2 = min(box2_height / 2, max_price2 - box2_price)
min_height2 = min(box2_height / 2, box2_price - min_price2)

max_height3 = min(box3_height / 2, max_price3 - box3_price)
min_height3 = min(box3_height / 2, box3_price - min_price3)

# Vertical lines for the first box
ax.plot([box_start1, box_start1], [box1_price - min_height1, box1_price + max_height1], color='turquoise', linestyle='--', label='Asset Purchase')
ax.plot([box_start1 + box_width1, box_start1 + box_width1], [box1_price - min_height1, box1_price + max_height1], color='grey', linestyle='--', label='Expiration Limit')

# Horizontal lines for the first box
ax.plot([box_start1, box_start1 + box_width1], [box1_price + max_height1, box1_price + max_height1], color='green', linestyle='--', label='Profit Taking Limit')
ax.plot([box_start1, box_start1 + box_width1], [box1_price - min_height1, box1_price - min_height1], color='red', linestyle='--', label='Stop Loss Limit')

# Vertical lines for the second box
ax.plot([box_start2, box_start2], [box2_price - min_height2, box2_price + max_height2], color='turquoise', linestyle='--')
ax.plot([box_start2 + box_width2, box_start2 + box_width2], [box2_price - min_height2, box2_price + max_height2], color='grey', linestyle='--')

# Horizontal lines for the second box (using the same colors and legend label as the first box)
ax.plot([box_start2, box_start2 + box_width2], [box2_price + max_height2, box2_price + max_height2], color='green', linestyle='--')
ax.plot([box_start2, box_start2 + box_width2], [box2_price - min_height2, box2_price - min_height2], color='red', linestyle='--')

# Vertical lines for the third box
ax.plot([box_start3, box_start3], [box3_price - min_height3, box3_price + max_height3], color='turquoise', linestyle='--')
ax.plot([box_start3 + box_width3, box_start3 + box_width3], [box3_price - min_height3, box3_price + max_height3], color='grey', linestyle='--')

# Horizontal lines for the third box (using the same colors and legend label as the first box)
ax.plot([box_start3, box_start3 + box_width3], [box3_price + max_height3, box3_price + max_height3], color='green', linestyle='--')
ax.plot([box_start3, box_start3 + box_width3], [box3_price - min_height3, box3_price - min_height3], color='red', linestyle='--')

# Add legend
ax.legend()
fig.tight_layout(pad = 1.5)

plt.savefig(path_default + '/visuals/TBM-Visualized.svg', format='svg') 

# Display the plot
plt.show()


# Descriptive Analysis

## General

In [ ]:
def calc_annualized_return_log(df):
    """
    Calculates the annualized log return for given assets.
    
    Args:
        df (dataframe): Dataframe of assets.
    Returns:
        float: annualized returns.
    """
    df.loc[:, 'log_return'] = np.log(df['close'] / df['close'].shift(1))
    total_log_return = df['log_return'].sum()
    average_daily_log_return = total_log_return / len(df)
    trading_days_per_year = 252
    annualized_return = average_daily_log_return * trading_days_per_year
    return annualized_return

In [ ]:
df_all = pd.read_csv(path_data_raw + '/trading_daily_all.csv')

df_all['date'] = pd.to_datetime(df_all['date'])

# Number of different stocks in our portfolio
df_all['cusip'].nunique()

In [ ]:
# Minimum and maximum timestamp
print(df_all['date'].min())
print(df_all['date'].max())

In [ ]:
grouped = df_all.groupby('cusip')['date']
time_duration = grouped.max() - grouped.min()

# Calculate the average time duration in days
average_time = time_duration.dt.days.mean()
print('AVG-Time', average_time)

# Convert average time duration to years and months
average_time_years = average_time // 365
average_time_months = (average_time % 365) // 30

print(average_time_years)
print(average_time_months)

In [ ]:
# Get number of entries per stock
entries_per_stock = df_all.groupby('cusip').count()

# Calculate the average number of entries per stock
average_entries = entries_per_stock['date'].mean()
average_entries

In [ ]:
df_all2 = df_all.set_index(['cusip', 'date']).sort_values(['cusip', 'date'])

In [ ]:
tokens = df_all2.index.get_level_values(0).unique() # Get all unique CUSIP values

volatilities = []

for token in tqdm(tokens):
    df_token = df_all2.loc[token] 
    
    # Calculate the daily percentage change
    df_token['daily_pct_change'] = df_token['close'].pct_change()

    # Calculate the average daily volatility
    average_volatility = df_token['daily_pct_change'].std()

    if not np.isnan(average_volatility):
        volatilities.append(average_volatility)

print('Average for stock portfolio:', np.mean(volatilities))

In [ ]:
# Comparison to the S&P 500
sp500 = request_yfinance_data('^GSPC', '2004-04-01', '2023-05-01')
russel3000 = request_yfinance_data('^RUA', '2004-04-01', '2023-05-01')

sp500['daily_pct_change'] = sp500['Close'].pct_change()
sp500_average_volatility = sp500['daily_pct_change'].std()
russel3000['daily_pct_change'] = russel3000['Close'].pct_change()
russel3000_average_volatility = russel3000['daily_pct_change'].std()
print(sp500_average_volatility)
print(russel3000_average_volatility)

In [ ]:
# Use MappingApi to get company information 
mappingApi = MappingApi(api_key=sec_api_key)

random_indexes = df_all.sample(n=10)

random_indexes = random_indexes.index

for idx in random_indexes:
    info = mappingApi.resolve('cusip', idx)
    print(info)

In [ ]:
# Descriptive Analysis Trading Daily Data
df = df_all

tokens = df.cusip.unique()
print(len(tokens))

In [ ]:
df = df.set_index(df['cusip'])
df = df.drop('cusip', axis=1)
df.head()

In [ ]:
annualized_returns = []

for token in tqdm(tokens):
    df_token = df.loc[token].copy()
    if len(df_token) > 2 and isinstance(df_token, pd.DataFrame):
        annualized_return = calc_annualized_return_log(df_token)
        annualized_returns.append(annualized_return)        
    else:
        continue

In [ ]:
annualized_returns_clean = annualized_returns[~np.isnan(annualized_returns)]
annualized_returns_clean = annualized_returns_clean[np.isfinite(annualized_returns_clean)]

In [ ]:
# Calculate summary statistics
mean = np.mean(annualized_returns_clean)
median = np.median(annualized_returns_clean)
std = np.std(annualized_returns_clean)
min_value = np.min(annualized_returns_clean)
max_value = np.max(annualized_returns_clean)

# Print the summary statistics
print("Mean:", mean)
print("Median:", median)
print("Standard Deviation:", std)
print("Minimum Value:", min_value)
print("Maximum Value:", max_value)

## BB-Strategy

In [ ]:
# Load the trades as determined by BB-Strategy
data = load_ml_data(path_data_combined, '/v6_bb_w_vert_barrier_and_external_features.csv', vertical_barrier=False)

# Processing
# Drop columns as they have too many nan values
data = data.drop(['feature_SP500_new_highs_change', 'feature_SP500_new_lows_change'], axis=1)

exclude_cols = ['feature_BTC', 'feature_CryptoMarketCap', 'feature_CryptoMarketCap_rolling_percentile', 'feature_BTC_rolling_percentile', 'feature_CryptoMarketCap_change', 'feature_BTC_change']

data.replace([np.inf, -np.inf], np.nan, inplace=True)

# Select columns to include by subtracting the exclude_cols from all columns
include_cols = data.columns.difference(exclude_cols)

# Drop rows with NaN values in include_cols
data = data.dropna(subset=include_cols, axis=0)

In [ ]:
df = data.copy(deep=True)
df[['feature_BTC', 'feature_CryptoMarketCap', 'feature_CryptoMarketCap_rolling_percentile', 'feature_BTC_rolling_percentile', 'feature_CryptoMarketCap_change', 'feature_BTC_change']] = df[['feature_BTC', 'feature_CryptoMarketCap', 'feature_CryptoMarketCap_rolling_percentile', 'feature_BTC_rolling_percentile', 'feature_CryptoMarketCap_change', 'feature_BTC_change']].fillna(0)
print(len(df))

In [ ]:
df['profit_rel'].describe()

In [ ]:
# Define the ranges
ranges = [-float('inf'), -1.0, -0.9, -0.8, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.15, -0.1, -0.05, 0.0, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 1200]
label_last = f"> {int(ranges[-2] * 100)}%"

# Create a new column 'range_category' based on the ranges
df['range_category'] = pd.cut(df['profit_rel'], ranges)

# Count the number of entries in each range
count_by_range = df['range_category'].value_counts().sort_index()

# Define custom labels for x-axis
labels = [f"-90% to -100%" if left == -1.0 else f"< {int(right * 100)}%" if left == -float('inf') else f"{int(left * 100)}% to {int(right * 100)}%" for left, right in zip(ranges[:-1], ranges[1:])]
labels[-1] = label_last

# Display the count for each range
for i, count in enumerate(count_by_range):
    print(f"Range {i+1}: {count}")

In [ ]:
# Extract the count values and range categories
counts = count_by_range.values
range_categories = count_by_range.index

fig, ax = plt.subplots(figsize=(15, 10))

# Create a bar plot with adjusted alignment
plt.bar(range(len(count_by_range)), count_by_range, align='center')  # Use 'align' parameter

# Set x-axis tick labels, rotate them by 45 degrees, and adjust alignment
plt.xticks(range(len(count_by_range)), labels, rotation=45, fontsize=18)  
plt.yticks(fontsize=18)  # Updated fontsize to 14

# Set axis labels and title
plt.xlabel('Relative Profit Range', fontsize=24)  
plt.ylabel('Number of Trades', fontsize=24)  
plt.title('Relative Profits Distribution', fontsize=24)  

# Adjust the layout to prevent label overlapping if necessary
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
# Calculate duration of trades
df['duration'] = df['t1'] - df['date']

# Calculate the average duration
average_duration = df['duration'].mean()

# Print the average duration
print(f"Average Duration: {average_duration}")

In [ ]:
# Find outliers
z_scores = (df['profit_rel'] - df['profit_rel'].mean()) / df['profit_rel'].std()
threshold = 3
outliers = df[abs(z_scores) > threshold]

In [ ]:
# Get statistics for outliers distinguised by profitable/unprofitable
print(outliers[outliers['bin'] == 0]['profit_rel'].mean())
print(len(outliers[outliers['bin'] == 0]))
print('-'*50)
print(outliers[outliers['bin'] == 1]['profit_rel'].mean())
print(len(outliers[outliers['bin'] == 1]))

## TBM-Strategy


In [ ]:
# Load trades as determined by TBM-Strategy
data = load_ml_data(path_data_combined, '/v1_tbm_and_external_features_with_fees.csv', vertical_barrier=True)

# Pre-Processing
# Drop columns as they have too many nan values
data = data.drop(['feature_SP500_new_highs_change', 'feature_SP500_new_lows_change'], axis=1)

exclude_cols = ['feature_BTC', 'feature_CryptoMarketCap', 'feature_CryptoMarketCap_rolling_percentile', 'feature_BTC_rolling_percentile', 'feature_CryptoMarketCap_change', 'feature_BTC_change']

data.replace([np.inf, -np.inf], np.nan, inplace=True)

# Select columns to include by subtracting the exclude_cols from all columns
include_cols = data.columns.difference(exclude_cols)

# Drop rows with NaN values in include_cols
data = data.dropna(subset=include_cols, axis=0)

df = data.copy(deep=True)
df[['feature_BTC', 'feature_CryptoMarketCap', 'feature_CryptoMarketCap_rolling_percentile', 'feature_BTC_rolling_percentile', 'feature_CryptoMarketCap_change', 'feature_BTC_change']] = df[['feature_BTC', 'feature_CryptoMarketCap', 'feature_CryptoMarketCap_rolling_percentile', 'feature_BTC_rolling_percentile', 'feature_CryptoMarketCap_change', 'feature_BTC_change']].fillna(0)

In [ ]:
# Check how many trades are short/long
print(len(df[df['side'] == -1]))
print(len(df[df['side'] == 1]))

In [ ]:
# Define the ranges
ranges = [-1, -0.9, -0.8, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.1, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 1200]
label_last = f"> {int(ranges[-2] * 100)}%"

# Create a new column 'range_category' based on the ranges
df['range_category'] = pd.cut(df['profit_rel'], ranges)

# Count the number of entries in each range for each side
count_by_range = df.groupby(['range_category', 'side']).size().unstack(fill_value=0).sort_index()

# Define custom labels for x-axis
labels = [f"{int(left * 100)}% to {int(right * 100)}%" if left != -1 else f"<={int(right * 100)}%" for left, right in zip(ranges[:-1], ranges[1:])]
labels[-1] = label_last

# Extract the count values and range categories for each side
counts_negative = count_by_range[-1].values
counts_positive = count_by_range[1].values
range_categories = count_by_range.index

fig, ax = plt.subplots(figsize=(15, 10))

# Create a bar plot for negative trades
plt.bar(range(len(count_by_range)), counts_negative, label='Short Trades', color="lightblue")

# Create a bar plot for positive trades
plt.bar(range(len(count_by_range)), counts_positive, label='Long Trades', color='darkblue', bottom=counts_negative)

# Set x-axis tick labels and rotate them by 45 degrees
plt.xticks(range(len(count_by_range)), labels, rotation=45, fontsize=17)
plt.yticks(fontsize=15)

# Set axis labels and title
plt.xlabel('Relative Profit Range', fontsize=20)
plt.ylabel('Number of Trades', fontsize=20)
plt.title('Relative Profits Distribution Triple-Barrier Method & Bollinger Bands', fontsize=20)

plt.ticklabel_format(style='plain', axis='y')

# Add a legend
plt.legend(fontsize=15)

# Adjust the layout to prevent label overlapping if necessary
plt.tight_layout()

# Save plot to file
plt.savefig(path_default + '/visuals/v1_TBM_trading_strat_distribution_rel_profits_by_side.svg', dpi='figure', format='svg')

# Display the plot
plt.show()


In [ ]:
# Realative profit over all trades
print(np.mean(df['profit_rel']) * 100)

print(df['profit_rel'].describe())

df['duration'] = df['t1'] - df['date']

# Calculate the average duration
average_duration = df['duration'].mean()
maximum_duration = df['duration'].max()
minimum_duration = df['duration'].min()


# Print the average duration
print(f"Average Duration: {average_duration}")
print(f"Maximum Duration: {maximum_duration}")
print(f"Minimum Duration: {minimum_duration}")

In [ ]:
# Outlier detection absolute profits
z_scores = np.abs((df['profit_abs'] - df['profit_abs'].mean()) / df['profit_abs'].std())
threshold = 3
outliers = df[z_scores > threshold]


# Get statistics for outliers distinguised by profitable/unprofitable
print(outliers[outliers['bin'] == 0]['profit_rel'].mean())
print(len(outliers[outliers['bin'] == 0]))
print('-'*50)
print(outliers[outliers['bin'] == 1]['profit_rel'].mean())
print(len(outliers[outliers['bin'] == 1]))

print(np.mean(outliers['profit_rel']))

print(len(outliers))

## Principal Component Analysis (PCA)

In [ ]:
def pca(X_scaled_ml, X_scaled_ho, k = 0.95):
    """
    Application of the Principal Component Analysis on the data (train and hold-out data).

    Args:
        X_scaled_ml (dataframe): Scaled training data.
        X_scaled_ho (dataframe): Scaled hold-out data.
        k (float, optional): Percentage of variance explained. Defaults to 0.95 (95%).
    Returns:
        Tuple: Tuple containing the dataframes for the scaled training and validation data.
    """

    pca_max = PCA(n_components = None, random_state = 42) # None will use all possible components
    pca_max.fit(X_scaled_ml)
    print('Variance explained by all components:', sum(pca_max.explained_variance_ratio_ *100))

    # Visualize the components and their importance
    plt.plot(np.cumsum(pca_max.explained_variance_ratio_ * 100))
    plt.xlabel('Number of Components')
    plt.ylabel('Explained Variance')
    plt.show()

    pca = PCA(n_components = k, random_state = 42)
    pca.fit(X_scaled_ml) # Fit on training data

    # Transform the data based on PCA (fitted on trainig data)
    X_ml_pca = pca.transform(X_scaled_ml)
    X_ho_pca = pca.transform(X_scaled_ho)

    # Create dataframe with the principal components
    X_ml_pca_df = pd.DataFrame(X_ml_pca, columns=[f'PC{i+1}' for i in range(X_ml_pca.shape[1])])
    X_ho_pca_df = pd.DataFrame(X_ho_pca, columns=[f'PC{i+1}' for i in range(X_ho_pca.shape[1])])

    return X_ml_pca_df, X_ho_pca_df

# Data Generation

## Basic Files

In [ ]:
# Read all CSV files (with OHLC data)
csv_files = [f for f in os.listdir(path_data_trading_daily) if f.endswith('.csv')]
csv_files = sorted(csv_files)
dfs = []

print('Reading and processing individual files ...')
# Loop through the CSV file names and read each file into a DataFrame
for file in tqdm(csv_files):
    file_path = os.path.join(path_data_trading_daily, file)
    df = pd.read_csv(file_path)

    # Add CUSIP to be able to group them later
    df.insert(0, 'cusip', str(file.split('.')[0]))

    # Rename columns to more meaningful names
    df = df.rename(columns = {
        'prcod': 'open',
        'prchd': 'high',
        'prccd': 'close',
        'prcld' : 'low',
        'cshtrd' : 'volume',
        'datadate': 'date'
    })

    df['date'] = pd.to_datetime(df['date'])
    df['date'] = df['date'].dt.strftime('%Y-%m-%d') # Format dat

    dfs.append(df)

combined_df = pd.concat(dfs)

combined_df.to_csv(path_data_raw + "/v1_trading_combined.csv", index = False)

## Features

In [ ]:
# Read the combined OHLC data
combined_data = read_data_csv()

combined_data = combined_data.set_index(['cusip', 'date']).sort_values(['cusip', 'date'])

tokens = combined_data.index.get_level_values(0).unique() # Get unique CUSIPs

tokens = sorted(tokens)

dfs_features = []
i = 0 # Counter used for buffering

for token in tqdm(tokens):
    # Get sub-dataframe with all entries for given token (CUSIP)
    df_token = combined_data.loc[token] 

    # Add technical features
    df_token = add_certain_ta_lib_features(df_token)

    # Add cusip and date again
    df_token['cusip'] = token
    df_token['date'] = pd.to_datetime(df_token.index)

    # Adding the finished assets
    dfs_features.append(df_token)

    i += 1

    # Buffering
    if i % 500 == 0:
        combined_data_features = pd.concat(dfs_features)
        combined_data_features = combined_data_features.set_index(['cusip', 'date']).sort_values(['cusip', 'date'])

        # Add CUSIP and date again
        combined_data_features.insert(0, 'date', pd.to_datetime(combined_data_features.index.get_level_values(1).values))
        combined_data_features.insert(0, 'cusip', combined_data_features.index.get_level_values(0))

        combined_data_features.to_csv(path_data_features + "/v1_features_part" + str(i) + '.csv', index = False)

        dfs_features = []

# Save last entries
combined_data_features = pd.concat(dfs_features)
combined_data_features = combined_data_features.set_index(['cusip', 'date']).sort_values(['cusip', 'date'])

# Add CUSIP and date again
combined_data_features.insert(0, 'date', pd.to_datetime(combined_data_features.index.get_level_values(1).values))
combined_data_features.insert(0, 'cusip', combined_data_features.index.get_level_values(0))

combined_data_features.to_csv(path_data_features + "/v1_features_part_last.csv", index = False)

In [ ]:
# Read all buffered feature files
csv_files_features = [f for f in os.listdir(path_data_features) if f.endswith('.csv')]
dfs_features = []

# Read files and concatenate
for file in tqdm(csv_files_features):
    file_path = os.path.join(path_data_features, file)
    df = pd.read_csv(file_path)
    dfs_features.append(df)

combined_dfs_features = pd.concat(dfs_features)

# Store all trades including their features into one file
combined_dfs_features.to_csv(path_data_features + "/v1_features_concatenated.csv", index=False)

## Trading Strategies

### BB-Strategy

In [ ]:
# Get the OHLC data for all assets
df_daily_trades = read_data_raw_csv(full_path = path_data_raw + '/trading_daily_all.csv')

In [ ]:
# Load the concatenated features for the BB-Strategy
df_combined = pd.read_csv(path_data_features + "/v1_features_concatenated.csv")

df_combined['date'] = pd.to_datetime(df_combined['date'].dt.strftime('%Y-%m-%d')) # format date to be set as index later

df_combined = df_combined.set_index(['cusip', 'date']).sort_values(['cusip', 'date'])

# Apply Trading Strategy Long-Only Bollinger Bands
tokens = df_combined.index.get_level_values(0).unique()

dfs_processed_bb = []

for token in tqdm(tokens):
    df_token = df_combined.loc[token]
    
    # Drop hurst as it has been identifed as irrelevant
    df_token = df_token.drop('feature_hurst', axis = 1)      

    df_daily_trades_token = df_daily_trades[df_daily_trades['cusip'] == str(token)]

    if len(df_daily_trades_token) == 0:
        print('ERROR: Could not find df_trading_daily for token.')

    df_processed = apply_bollinger_bands_v2(df_daily_trades_token, df_token)

    if len(df_processed) > 0:
            # Set CUSIP and date again
            df_processed.loc[:, 'cusip'] = str(token)
            df_processed.loc[:, 'date'] = pd.to_datetime(df_processed.index.values)
            dfs_processed_bb.append(df_processed)
    else:
        continue
    
    # Save last entries
    data_f = pd.concat(dfs_processed_bb)
    data_f = data_f.set_index(['cusip', 'date']).sort_values(['cusip', 'date'])
    data_f = data_f.dropna()

    # Set CUSIP and date again
    data_f['cusip'] = data_f.index.get_level_values(0)
    data_f['date'] = pd.to_datetime(data_f.index.get_level_values(1).values)
    
    data_f.to_csv(path_data_strategy + '/v3_w_vert_barrier_applied_bb_.csv', index=False)



In [ ]:
csv_files_features = [f for f in os.listdir(path_data_strategy) if f.startswith('v3_w_vert_barrier')]

dfs_features = []

for file in tqdm(csv_files_features):
    file = file.lstrip('._') # Resolve file system issue
    file_path = os.path.join(path_data_features, file)
    df = load_ml_data(path_data_strategy, file, vertical_barrier=False)
    dfs_features.append(df)

# Combine all individual files with trades as determined by strategy
combined_dfs_features = pd.concat(dfs_features)

combined_dfs_features.to_csv(path_data_strategy + "/v6_BB_w_vert_barrier_all.csv", index=False)

### TBM-Strategy

In [ ]:
# Apply Trading Strategy TBM
combined_dfs_features = pd.read_csv(path_data_features + "/v1_features_concatenated.csv")

combined_dfs_features = combined_dfs_features.set_index(['cusip', 'date']).sort_values(['cusip', 'date'])

tokens = combined_dfs_features.index.get_level_values(0).unique() # Get unique CUSIPs

tokens = sorted(tokens)

index = 0 # Counter for Buffering

dfs_processed_tbm = []
for token in tqdm(tokens):
    df_token = combined_dfs_features.loc[token]

    # Drop hurst as it has been identifed as irrelevant
    df_token = df_token.drop('feature_hurst', axis=1)

    df_processed = apply_tbm(df_token, 0.0, 10, 2, 2) # Vertical-barrier: 10 days, pt=2, sl=2

    # Set CUSIP and date
    df_processed['cusip'] = token
    df_processed['date'] = pd.to_datetime(df_processed.index.values)

    dfs_processed_tbm.append(df_processed)
    index += 1

    if index % 500 == 0: # Buffering
        data_f = pd.concat(dfs_processed_tbm)
        data_f = data_f.set_index(['cusip', 'date']).sort_values(['cusip', 'date'])
        data_f = data_f.dropna()

        # Set CUSIP and date
        data_f['cusip'] = data_f.index.get_level_values(0)
        data_f['date'] = pd.to_datetime(data_f.index.get_level_values(1).values)

        data_f.to_csv(path_data_strategy + "/v1_applied_tbm_part" + str(index) + '.csv', index = False)
        dfs_processed_tbm = []

# Saving last entries 
data_f = pd.concat(dfs_processed_tbm)
data_f = data_f.set_index(['cusip', 'date']).sort_values(['cusip', 'date'])
data_f = data_f.dropna()

# Set CUSIP and date
data_f['cusip'] = data_f.index.get_level_values(0)
data_f['date'] = pd.to_datetime(data_f.index.get_level_values(1).values)

data_f.to_csv(path_data_strategy + '/v1_applied_tbm_part_last.csv', index = False)

In [ ]:
# Combine all individual files
csv_files_tbm = [f for f in os.listdir(path_data_strategy) if f.endswith('.csv')]
dfs_tbm = []

for file in tqdm(csv_files_tbm):
    file_path = os.path.join(path_data_strategy, file)
    df = pd.read_csv(file_path)
    dfs_tbm.append(df)

data_tbm_final = pd.concat(dfs_tbm)

data_tbm_final.to_csv(path_data_strategy + '/v1_applied_tbm_full.csv', index=False) # saving to csv

## External Data (FRED & yFinance)

In [ ]:
# Set end date for external data is now
end_date = datetime.datetime.now().strftime('%Y-%m-%d')
start_date = '2000-01-01'

# Get FRED and yFinance data
data_fred = get_fred_data(start_date, end_date)
data_yfinance = get_yfinance_data(start_date, end_date)

# Concat both data
data_apis = pd.concat([data_fred, data_yfinance], axis=1)

# Add date column for reference later
data_apis.insert(0, 'date', data_apis.index) 

# Drop all nans besides BTC and CryptoMarketCap as those have many (data starts in 2017)
data_apis.dropna(subset=data_apis.columns.difference(['feature_BTC', 'feature_CryptoMarketCap']), inplace = True) 

# Get all feature columns
feature_columns = [col for col in data_apis.columns if col.startswith('feature_')]

# Comput percentile and change for all feature columns
for feature in tqdm(feature_columns):
    data = compute_percentile_and_change(data_apis, feature)

# Drop all nans besides BTC and CryptoMarketCap as those have many (data starts in 2017)
data.dropna(subset=data_apis.columns.difference(['feature_BTC', 'feature_CryptoMarketCap']), inplace=True) 

# Store external data to csv
data.to_csv(path_data_external + '/fred_yfinance_percentiles_and_changes.csv', index=False) # saving to csv

## Combine Trading and External Data

In [ ]:
# Reading trades and external data
df_trading = load_ml_data(path_data_strategy, 'bb/v6_BB_w_vert_barrier_all.csv', vertical_barrier=False) # same for tbm-strat
df_external = pd.read_csv(path_data_external + '/fred_yfinance_data_percentiles_and_changes.csv')

# Set index for both datasets
df_trading = df_trading.set_index(pd.to_datetime(df_trading['date']))
df_external = df_external.set_index(pd.to_datetime(df_external['date']))

In [ ]:
df_trading = df_trading.drop('date', axis=1)
df_external = df_external.drop('date', axis=1)
df_trading['cusip'] = df_trading['cusip'].astype(str)

In [ ]:
# Merge trades with external data based on date
joined_df = df_trading.merge(df_external, left_index=True, right_index=True)

joined_df['date'] = pd.to_datetime(joined_df.index)

joined_df.to_csv(path_data_combined + '/v6_bb_w_vert_barrier_and_external_features.csv', index=False) # saving to csv

## NaN Processing

In [ ]:
# Load data (here for BB-Stratey)
# For TBM-Strategy it is done in the same way
data = load_ml_data(path_data_combined, '/v6_bb_w_vert_barrier_and_external_features.csv', vertical_barrier=False)
data.head(3)

In [ ]:
nans = data.isna().sum()
nans

In [ ]:
# Drop columns as they have too many nan values
data = data.drop(['feature_SP500_new_highs_change', 'feature_SP500_new_lows_change'], axis = 1)

In [ ]:
exclude_cols = ['feature_BTC', 'feature_CryptoMarketCap', 'feature_CryptoMarketCap_rolling_percentile', 'feature_BTC_rolling_percentile', 'feature_CryptoMarketCap_change', 'feature_BTC_change']

data.replace([np.inf, -np.inf], np.nan, inplace=True)

# Select columns to include by subtracting the exclude_cols from all columns
include_cols = data.columns.difference(exclude_cols)

# Drop rows with NaN values in include_cols
data = data.dropna(subset=include_cols, axis=0)

## Partition the Data

In [ ]:
df = data.copy(deep=True) # Use the data from the nan pre-processing step

# Fill NaN values in Crypto data with 0
df[['feature_BTC', 'feature_CryptoMarketCap', 'feature_CryptoMarketCap_rolling_percentile', 'feature_BTC_rolling_percentile', 'feature_CryptoMarketCap_change', 'feature_BTC_change']] = df[['feature_BTC', 'feature_CryptoMarketCap', 'feature_CryptoMarketCap_rolling_percentile', 'feature_BTC_rolling_percentile', 'feature_CryptoMarketCap_change', 'feature_BTC_change']].fillna(0)

# Set holdout date used for out-of-sample dataset
hold_out_date = pd.to_datetime('2020-01-01')

# Set for training, validation and testing
df_selected_ml = df_selected[df_selected['date'] <= pd.to_datetime(hold_out_date)]
df_selected_ho = df_selected[df_selected['date'] > pd.to_datetime(hold_out_date)]

# Reset Indexes
df_selected_ml = df_selected_ml.reset_index(drop=True)
df_selected_ho = df_selected_ho.reset_index(drop=True)


#################################################################################################
#################################################################################################

# Scalers
scaler_standard = StandardScaler()
scaler_minmax = MinMaxScaler()
scaler_robust = RobustScaler()

# Feature Columns
subset_cols = df_selected.filter(like='feature_').columns

# Get Feature Columns Subset
df_ml_features = df_selected_ml[subset_cols]
df_ho_features = df_selected_ho[subset_cols]

# Standard Scaler
df_ml_features_ss = pd.DataFrame(scaler_standard.fit_transform(df_ml_features), columns=df_ml_features.columns)
df_ho_features_ss = pd.DataFrame(scaler_standard.transform(df_ho_features), columns=df_ho_features.columns)

# Min-Max Scaler
df_ml_features_mms = pd.DataFrame(scaler_minmax.fit_transform(df_ml_features), columns=df_ml_features.columns)
df_ho_features_mms = pd.DataFrame(scaler_minmax.transform(df_ho_features), columns=df_ho_features.columns)

# Robust Scaler
df_ml_features_rob = pd.DataFrame(scaler_robust.fit_transform(df_ml_features), columns=df_ml_features.columns)
df_ho_features_rob = pd.DataFrame(scaler_robust.transform(df_ho_features), columns=df_ho_features.columns)

# Apply PCA
df_ml_features_pca_ss, df_ho_features_pca_ss = pca(df_ml_features_ss, df_ho_features_ss)
df_ml_features_pca_mms, df_ho_features_pca_mms = pca(df_ml_features_mms, df_ho_features_mms)
df_ml_features_pca_rob, df_ho_features_pca_rob = pca(df_ml_features_rob, df_ho_features_rob)

print('#Training', len(df_selected_ml))
print('#Testing', len(df_selected_ho))
print('#Features', len(df_selected_ml.columns))

In [ ]:
# Save all of the subsets of the data so we have consistency in our model training
df_selected_ml.to_csv(path_data_partitioned_bb_with_fees_train + '/v1_df_ml.csv', index = False)
df_selected_ho.to_csv(path_data_partitioned_bb_with_fees_test + '/v1_df_ho.csv', index = False)

# Standard
df_ml_features_ss.to_csv(path_data_partitioned_bb_with_fees_scaled + '/v1_df_ml_features_scaled_ss.csv', index = False)
df_ho_features_ss.to_csv(path_data_partitioned_bb_with_fees_scaled + '/v1_df_ho_features_scaled_ss.csv', index = False)

# MinMax
df_ml_features_mms.to_csv(path_data_partitioned_bb_with_fees_scaled + '/v1_df_ml_features_scaled_mms.csv', index = False)
df_ho_features_mms.to_csv(path_data_partitioned_bb_with_fees_scaled + '/v1_df_ho_features_scaled_mms.csv', index = False)

# Robust
df_ml_features_rob.to_csv(path_data_partitioned_bb_with_fees_scaled + '/v1_df_ml_features_scaled_rob.csv', index = False)
df_ho_features_rob.to_csv(path_data_partitioned_bb_with_fees_scaled + '/v1_df_ho_features_scaled_rob.csv', index = False)

# PCA
df_ml_features_pca_ss.to_csv(path_data_partitioned_bb_with_fees_pca + '/v1_df_ml_features_pca_ss.csv', index = False)
df_ho_features_pca_ss.to_csv(path_data_partitioned_bb_with_fees_pca + '/v1_df_ho_features_pca_ss.csv', index = False)
df_ml_features_pca_mms.to_csv(path_data_partitioned_bb_with_fees_pca + '/v1_df_ml_features_pca_mms.csv', index = False)
df_ho_features_pca_mms.to_csv(path_data_partitioned_bb_with_fees_pca + '/v1_df_ho_features_pca_mms.csv', index = False)
df_ml_features_pca_rob.to_csv(path_data_partitioned_bb_with_fees_pca + '/v1_df_ml_features_pca_rob.csv', index = False)
df_ho_features_pca_rob.to_csv(path_data_partitioned_bb_with_fees_pca + '/v1_df_ho_features_pca_rob.csv', index = False)

In [ ]:
# Check if the partitioning worked as expected
df_1 = load_ml_data(path_data_partitioned_bb_with_fees_train, 'v1_df_ml.csv', vertical_barrier=False)
df_2 = load_ml_data(path_data_partitioned_bb_with_fees_test, 'v1_df_ho.csv', vertical_barrier=False)

duplicate_rows = df_1[df_1.duplicated(subset=['cusip', 'date'], keep=False)]
print(len(duplicate_rows))

# Model Training

## BB-Strategy

### XGBoost

In [ ]:
def objective(trial):
    """
    This function is used to evaluate the objective function for hyperparameter optimization using Optuna library.
    
    Args:
        trial (instance of Optuna trial class): An instance of the Optuna Trial class.
    
    Returns:
        Float: The mean log loss score.
    """
    
    # Load the training data
    df_train = load_ml_data(path_data_partitioned_bb_with_fees_train, 'v1_df_ml.csv', vertical_barrier=False)
    
    # Split the features and target variables
    X = df_train.filter(like='feature_')
    y = df_train['bin']
    
    # Define the cross-validation strategy
    cv = KFold(n_splits=3, shuffle=True, random_state=42)
    
    # Array to store cross-validation scores
    cv_scores = np.empty(3)
    
    # Define the hyperparameters to optimize
    params = {
        'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4, 0.5, 0.6, 0.7, 0.8, 1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02]),
        'max_depth': trial.suggest_categorical('max_depth', [5, 7, 9, 11, 13, 15, 17]),
        'random_state': 42,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
        'gamma': trial.suggest_float("gamma", 0.0, 1.0),
        'grow_policy': trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
        'nthread': 8
    }
    
    # Perform cross-validation
    for idx, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        # Create and train the XGBoost model
        model = xgb.XGBClassifier(**params)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=100, verbose=False)
        
        # Make predictions on the validation set
        preds = model.predict(X_val)
        
        # Calculate the log loss error
        error = log_loss(y_val, preds)
        cv_scores[idx] = error
        
    # Return the mean of the log loss
    return np.mean(cv_scores)

In [ ]:
# Create Optuna study with the objective to minimize the log loss
study = optuna.create_study(direction="minimize", study_name="XGBoost BB-Strategy")
study.optimize(objective, n_trials=100)
print(study.best_trial)

# Print best hyperparameter combination
print(f"\tBest value (precision): {study.best_value:.5f}")
print(f"\tBest params:")
for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

### LGBM

In [ ]:
def objective(trial):
    """
    This function is used to evaluate the objective function for hyperparameter optimization using Optuna library.
    
    Args:
        trial (instance of Optuna trial class): An instance of the Optuna Trial class.
    
    Returns:
        Float: The mean log loss score.
    """
    
    # Load the ML data
    df_train = load_ml_data(path_data_partitioned_bb_with_fees_train, 'v1_df_ml.csv', vertical_barrier=False)
    X = df_train.filter(like='feature_')
    y = df_train['bin']
    
    # Create cross-validation instance
    cv = KFold(n_splits=3, shuffle=True, random_state=42)
    cv_scores = np.empty(3)

    # Set the parameters for LGBM model
    param = {
        "objective": "binary",
        "metric": "average_precision",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_threads": 8,
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 20),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_depth": trial.suggest_int("max_depth", 2, 15),
        "max_bin": trial.suggest_int("max_bin", 10, 255),
        "learning_rate": trial.suggest_float("learning_rate", 1e-8, 1.0, log=True),
    } 
        
    for idx, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        # Load the train and validation data
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        # Create LGBM datasets
        dtrain = lgb.Dataset(X_train, label=y_train)
        dvalid = lgb.Dataset(X_val, label=y_val)
        
        # Define pruning callback for early stopping
        pruning_callback = LightGBMPruningCallback(trial, "binary_logloss")

        # Train the LGBM model
        gbm = lgb.train(param, dtrain, valid_sets=[dvalid], callbacks=[pruning_callback])
        
        # Make predictions and calculate precision score
        preds = gbm.predict(X_val)
        pred_labels = np.rint(preds)

        # Calculate and store the error
        error = log_loss(pred_labels, preds)
        cv_scores[idx] = error

    # Return the mean precision score
    return np.mean(cv_scores)

In [ ]:
# Create Optuna study with the objective to maximize the precision
study = optuna.create_study(direction="maximize", study_name='LGBM BB-Strategy')
study.optimize(objective, n_trials=300)

# Print hyperparameter optimization results
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")

trial = study.best_trial
print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

### ANN

In [ ]:
# Set seed for PyTorch to ensure reproducibility
torch.manual_seed(42)

## Load the data
train = load_ml_data(path_data_partitioned_bb_with_fees_train, 'v1_df_ml.csv', vertical_barrier = False)
train_features = pd.read_csv(path_data_partitioned_bb_with_fees_scaled + '/v1_df_ml_features_scaled_ss.csv') # Features
train_target = train['bin'] # Targets

# Set hyperparameters
learning_rate = 0.0001
bs = 256
epochs = 250

# Create path details
now = datetime.datetime.now()
timestamp = now.strftime("%Y-%m-%d-%H-%M-%S")
execution_path = path_data_results_bb_nn + '/' + timestamp

try:
    os.mkdir(execution_path)
except Exception as e:
    print(e)
    print('Could not create path')

# Define config that is later stored
config = {
    'Date' : str(timestamp),
    'train_set': str(path_data_partitioned_bb_with_fees_train),
    'train_features': 'features-scaled-ss',
    'test_set': str(path_data_partitioned_bb_with_fees_test),
    'learning_rate': str(learning_rate),
    'batch_size': str(bs),
    'epochs': str(epochs),
}

# Create loss function, model, and optimizer
loss_fn = nn.BCELoss()
model = CustomMLP_BB(input_shape=train_features.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

structure_dict = get_structure_dict(model)

config['network_structure'] = structure_dict
config['loss'] = str(loss_fn)
config['optimizer'] = str(optimizer)

save_config(execution_path, config) # Save config to filesystem

X_train, X_val, y_train, y_val = train_test_split(
    train_features, train_target, test_size=0.2, random_state=4
)
print('#Training Examples', len(X_train))
print('#Validation Examples', len(X_val))

wandb.init(
    project = 'master-thesis-nn',
    entity= 'lars-s7',
    name=f"Neural_Network_Execution-Path_{execution_path}", 
    config = config,
    reinit=True
)

# Datasets
train_dataset = CustomStockDataSet(X_train, y_train)
val_dataset = CustomStockDataSet(X_val, y_val)

# Data Loader
train_dl = torch.utils.data.DataLoader(train_dataset, batch_size=bs, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_dataset, batch_size=bs, shuffle=False)

# Train neural network using pre-defined method
_, _ = train_model(model, train_dl, val_dl, epochs, optimizer, loss_fn, 0, execution_path)

## TBM-Strategy

### XGBoost

In [ ]:
def objective(trial):
    """
    This function is used to evaluate the objective function for hyperparameter optimization using Optuna library.
    
    Args:
        trial (instance of Optuna trial class): An instance of the Optuna Trial class.
    
    Returns:
        Float: The mean log loss score.
    """
    # Load the ML data
    df_train = load_ml_data(path_data_partitioned_tbm_with_fees_train, 'v1_df_ml.csv', vertical_barrier = True)
    X = df_train.filter(like='feature_') # Features
    y = df_train['bin'] # Targets
    
    # Create k-fold object
    cv = KFold(n_splits=3, shuffle=True, random_state=42)
    cv_scores_loss = np.empty(3)

    # Define hyperparameter space
    params = {
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'random_state': 42,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
        'gamma' : trial.suggest_float("gamma", 0.0, 1.0),
        'grow_policy' : trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
        'nthread': 8
    }
        
    for idx, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        # Create model and train it
        model = xgb.XGBClassifier(**params)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=100, verbose=False)
        preds = model.predict(X_val)
        
        error = log_loss(y_val, preds)
        cv_scores_loss[idx] = error

    # Return the mean log loss   
    return np.mean(cv_scores_loss)

In [ ]:
# Create Optuna study that has the objective of minimizing the log loss
study = optuna.create_study(direction="minimize", study_name="XGBoost TBM-Strategy")
study.optimize(objective, n_trials=100)
print(study.best_trial)

print(f"\tBest value (precision): {study.best_value:.5f}")
print(f"\tBest params:")
for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

### LGBM

In [ ]:
def objective(trial):
    """
    This function is used to evaluate the objective function for hyperparameter optimization using Optuna library.
    
    Args:
        trial (instance of Optuna trial class): An instance of the Optuna Trial class.
    
    Returns:
        Float: The mean log loss score.
    """
    # Load the data
    df_train = load_ml_data(path_data_partitioned_tbm_with_fees_train, 'v1_df_ml.csv', vertical_barrier = True)
    df_train_features = df_train.filter(like='feature_') # Features
    df_train_target = df_train['bin'] # Targets

    X = df_train_features
    y = df_train_target
    
    # Create k-fold object
    cv = KFold(n_splits=3, shuffle=True, random_state=42)
    cv_scores_loss = np.empty(3)
    
    # Define hyperparameter space
    params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_threads": 4,
        
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 20),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_depth": trial.suggest_int("max_depth", 2, 15),
        "max_bin": trial.suggest_int("max_bin", 10, 255),
        "learning_rate": trial.suggest_float("learning_rate", 1e-8, 1.0, log=True)
    } 
    
    config = dict(trial.params)
    config['trial_number'] = trial.number
    
    for idx, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        # Get data for fold
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        dtrain = lgb.Dataset(X_train, label=y_train)
        dvalid = lgb.Dataset(X_val, label=y_val)
        
        pruning_callback = LightGBMPruningCallback(trial, "binary_logloss")
        gbm = lgb.train(params, dtrain, valid_sets=[dvalid], callbacks=[pruning_callback])
        
        preds = gbm.predict(X_val)
        pred_labels = np.rint(preds)
        
        loss = log_loss(y_val, pred_labels)        
        cv_scores_loss[idx] = loss
    return np.mean(cv_scores_loss)

In [ ]:
# Create study that minimized the log loss
study = optuna.create_study(direction="minimize", study_name="LGBM TBM-Strategy")
    
study.optimize(objective, n_trials=300)

print(f"\tBest params:")
for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

### ANN

In [ ]:
# Set random seed for reproducibility
torch.manual_seed(42)

# Load the data
train = load_ml_data(path_data_partitioned_tbm_with_fees_train, 'v1_df_ml.csv', vertical_barrier = True)
train_features = pd.read_csv(path_data_partitioned_tbm_with_fees_scaled + '/v1_df_ml_features_scaled_ss.csv') # Features
train_target = train['bin'] # Targets

# Hyperparameters
learning_rate = 0.0001
epochs = 350
batch_size = 256

# Create execution path
now = datetime.datetime.now()
timestamp = now.strftime("%Y-%m-%d-%H-%M-%S")
execution_path = path_data_results_tbm_nn + '/' + timestamp

try:
    os.mkdir(execution_path)
except Exception as e:
    print(e)
    print('Could not create path')

config = {
    'Date' : str(timestamp),
    'train_set': 'Standard Scaler',
    'test_set': 'Standard Scaler',
    'learning_rate': str(learning_rate),
    'epochs': str(epochs),
}

# Define model, loss function, and optimizer
loss_fn = nn.BCELoss()
model = CustomMLP(input_shape=train_features.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

structure_dict = get_structure_dict(model)

config['network_structure'] = structure_dict
config['loss'] = str(loss_fn)
config['optimizer'] = str(optimizer)

save_config(execution_path, config)

X_train, X_val, y_train, y_val = train_test_split(
    train_features, train_target, test_size=0.2, random_state=4
)

print('#Training Examples', len(X_train))
print('#Validation Examples', len(X_val))

wandb.init(
    project = 'master-thesis-nn',
    entity= 'lars-s7',
    name=f"TBM_Neural_Network_Execution-Path_{execution_path}", 
    config = config,
    reinit=True
)

# Datasets
train_dataset = CustomStockDataSet(X_train, y_train)
val_dataset = CustomStockDataSet(X_val, y_val)

# Data Loader
train_dl = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Train neural network
_, _ = train_model(model, train_dl, val_dl, epochs, optimizer, loss_fn, 0, execution_path)


### Recursive Feature Elimination (RFE)



In [ ]:
# Train
df_train = load_ml_data(path_data_partitioned_bb_with_fees_train, 'v1_df_ml.csv', vertical_barrier = False)
df_train_features = df_train.filter(like='feature_')
df_train_target = df_train['bin']

# Test
df_test = load_ml_data(path_data_partitioned_bb_with_fees_test, 'v1_df_ho.csv', vertical_barrier = False)
df_test_features = df_test.filter(like='feature_')
df_test_target = df_test['bin']

# Config for LGBM (based on Optuna optimization)
params_lgbm = {
    "num_threads": 8,
    "boosting_type": "gbdt",
    "lambda_l1": 2.8269687100646643,
    "lambda_l2": 0.1132729746179371,
    "num_leaves": 222,
    "feature_fraction": 0.6029600980261194,
    "bagging_fraction": 0.9924346775670325,
    "bagging_freq": 2,
    "min_child_samples": 65,
    "max_depth": 14,
    "max_bin": 136,
    "learning_rate": 0.30197097330510875
}

# Initialize LGBM and set params
clf_lgbm = lgb.LGBMClassifier(seed=42)
clf_lgbm.set_params(**params_lgbm)

cv = KFold(n_splits=5, random_state=42, shuffle=True)

rfecv = RFECV(
    estimator=clf_lgbm,
    step=1,
    cv=cv,
    scoring="precision",
    min_features_to_select=1,
)

rfecv.fit(df_train_features, df_train_target)

In [ ]:
# Access the feature importances from the underlying decision tree model
feature_importances = rfecv.estimator_.feature_importances_

# Create a list of feature-importance pairs
importance_pairs = list(zip(df_train_features.columns, feature_importances))

# Sort the list based on importance values in descending order
importance_pairs.sort(key=lambda x: x[1], reverse=True)

# Print the feature importances in descending order
for feature, importance in importance_pairs:
    print(f"Feature: {feature}, Importance: {importance}")
    
sorted_features1, sorted_importances1 = zip(*importance_pairs[:10])
sorted_features2, sorted_importances2 = zip(*importance_pairs[10:20])

In [ ]:
# Plot the feature importances and save them to the filesystem
plt.figure(figsize=(10, 8))
plt.bar(sorted_features1, sorted_importances1)
plt.xlabel('Top 10 Features', fontsize=15)
plt.ylabel('Feature Importance', fontsize=15)
plt.title('BB-Strategy Feature Importances', fontsize=15)
plt.xticks(rotation=90, fontsize=15)
plt.yticks(fontsize=15) 
plt.tight_layout()
plt.savefig(path_default + '/visuals/v1_unscaled_with_fees_RFE_feature_importance_top_20_part1.svg', format='svg', dpi='figure')
plt.show()

# Buy & Hold Dataset Generation

## BB-Strategy

In [ ]:
df_trading_daily = read_data_raw_csv() # Get initial raw data

df_bb = load_ml_data(path_data_partitioned_bb_with_fees_train, 'v1_df_ml.csv', vertical_barrier=False) # Get all BB trades in out-of-sample dataset

In [ ]:
cusips = df_bb['cusip'].unique().tolist() # Unique CUSIP identifiers
print('# Unique CUSIPs', len(cusips))

i = 0 # Counter for buffering
rows = [] # Store buy-and-hold performance for each CUSIP

for cusip in tqdm(cusips):
    # Get first buying entry of BB-Strategy
    buying_entry = df_bb[df_bb['cusip'] == cusip].nsmallest(1, 'date')

    # Get latest available data point for given CUSIP
    selling_entry = df_trading_daily[df_trading_daily['cusip'] == cusip].nlargest(1, 'date')
    
    # Check if buying entry and selling entry could be obtained
    if buying_entry is not None and selling_entry is not None:
        buying_price = buying_entry['close'].iloc[0] # Buy price when first BB-Strategy trade was initiated
        selling_price = selling_entry['close'].iloc[0] # Selling price is last available data point
        relative_profit = (selling_price - buying_price) / buying_price

        # Create entry for trade
        entry = {
            'cusip': str(cusip),
            'side': 1,
            'date': buying_entry['date'].iloc[0],
            't1': selling_entry['date'].iloc[0],
            'close': buying_price,
            't1_price': selling_price,
            'profit_rel': relative_profit,
            'bin': 1 if relative_profit > 0 else 0
        }
        rows.append(entry)
    else:
        print('Buying-Date or Selling-Date not found!')
    i += 1
    
    # Buffering
    if i > 0 and i % 500 == 0:
        buy_and_hold_data = pd.DataFrame(rows)
        buy_and_hold_data.to_csv(path_data_buy_hold + '/v1_BB_buy_and_hold_' + str(i) + '.csv', index = False)
        rows = []
buy_and_hold_data = pd.DataFrame(rows)
buy_and_hold_data.to_csv(path_data_buy_hold + '/v1_BB_buy_and_hold_' + str(i) + '.csv', index = False)

In [ ]:
# Combine individual files that were created because of buffering
csv_files_features = [f for f in os.listdir(path_data_buy_hold) if f.startswith('v1_BB_')]
dfs_features = []

for file in tqdm(csv_files_features):
    file_path = os.path.join(path_data_features, file)
    df = load_ml_data(path_data_buy_hold, file, vertical_barrier=False)
    dfs_features.append(df)

combined_buy_n_hold = pd.concat(dfs_features)

# Store single dataframe with all buy-and-hold performances
combined_buy_n_hold.to_csv(path_data_buy_hold + "/v1_BB_buy_and_hold_combined.csv", index=False)

## TBM-Strategy

In [ ]:
df_trading_daily = read_data_raw_csv() # Get raw asset data

df_tbm = load_ml_data(path_data_partitioned_tbm_with_fees_test, 'v1_df_ho.csv', vertical_barrier=False) # Get trades as determined by TBM-Strategy

In [ ]:
cusips = df_tbm['cusip'].unique().tolist() # Get unique CUSIPs
print('# Unique CUSIPs', len(cusips))

i = 0 # Counter for buffering
rows = []

for cusip in tqdm(cusips):
    # Earliest trade entry as determined by the TBM-Strategy
    buying_entry = df_tbm[df_tbm['cusip'] == cusip].nsmallest(1, 'date')
    # Latest data point available
    selling_entry = df_trading_daily[df_trading_daily['cusip'] == cusip].nlargest(1, 'date')
    
    if buying_entry is not None and selling_entry is not None:
        buying_price = buying_entry['close'].iloc[0]
        selling_price = selling_entry['close'].iloc[0]
        relative_profit = (selling_price - buying_price) / buying_price

        # Create buy-and-hold entry
        entry = {
            'cusip': str(cusip),
            'side': 1,
            'date': buying_entry['date'].iloc[0],
            't1': selling_entry['date'].iloc[0],
            'close': buying_price,
            't1_price': selling_price,
            'profit_rel': relative_profit,
            'bin': 1 if relative_profit > 0 else 0
        }

        rows.append(entry)
    else:
        print('Buying-Date or Selling-Date not found!')
    i += 1
    
    # Buffering
    if i > 0 and i % 500 == 0:
        buy_and_hold_data = pd.DataFrame(rows)
        buy_and_hold_data.to_csv(path_data_buy_hold + '/v1_TBM_buy_and_hold_' + str(i) + '.csv', index = False)
        rows = []
        
buy_and_hold_data = pd.DataFrame(rows)
buy_and_hold_data.to_csv(path_data_buy_hold + '/v1_TBM_buy_and_hold_' + str(i) + '.csv', index = False)

In [ ]:
# Combine individual files due to buffering
csv_files_features = [f for f in os.listdir(path_data_buy_hold) if f.startswith('v1_TBM_')]
dfs_features = []

for file in tqdm(csv_files_features):
    file_path = os.path.join(path_data_features, file)
    df = load_ml_data(path_data_buy_hold, file, vertical_barrier=False)
    dfs_features.append(df)

combined_buy_n_hold = pd.concat(dfs_features)

# Save combined file
combined_buy_n_hold.to_csv(path_data_buy_hold + "/v1_TBM_buy_and_hold_combined.csv", index=False)

# Sanity Check (10 Assets) BB-Strategy

In the following a sanity check for 10 assets is conducted. Hereby, 10 stocks are chosen and evaluated. Thus, Meta-Labeling is tested.

In [ ]:
# Get 10 Stock from the data
# Read Data
########## Datasets ##########
# Train
df_train = load_ml_data(path_data_partitioned_bb_with_fees_train, 'v1_df_ml.csv', vertical_barrier = False)
df_train_features = df_train.filter(like='feature_')
df_train_target = df_train['bin']

# Test
df_test = load_ml_data(path_data_partitioned_bb_with_fees_test, 'v1_df_ho.csv', vertical_barrier = False)
df_test_features = df_test.filter(like='feature_')
df_test_target = df_test['bin']
########## Datasets ##########

In [ ]:
mdl_loaded = load_model_sklearn(path_data_final_models, 'BB_LGBM') # Load LGBM model

In [ ]:
pred = mdl_loaded.predict(df_test_features) # predict

In [ ]:
calc_full_evaluation_w_prediction(pred, df_test, df_test_features, df_test_target)

In [ ]:
df_test.sample(n=10).cusip # Get 10 random CUSIPs

# The CUSIPs that were output by the previous line of code
cusips_sanity = [
    '67092P102',
    '683797104',
    '70399K933',
    '72201R304',
    '72201R775',
    '767292105',
    '78440X507',
    '829214105',
    '90187B606',
    '90214Q766'
]

In [ ]:
df_sanity_trades = df_test[df_test['cusip'].isin(cusips_sanity)]

df_sanity_trades_features = df_sanity_trades.filter(like='feature_') # Features
df_sanity_trades_target = df_sanity_trades['bin'] # Targets

prediction = mdl_loaded.predict(df_sanity_trades_features) # Predict only our selected CUSIPs

calc_full_evaluation_w_prediction(prediction, df_sanity_trades, df_sanity_trades_features, df_sanity_trades_target)

In [ ]:
full_financial_performance_eval(df_sanity_trades.copy(deep=True), prediction, russel=True)
full_financial_performance_eval(df_sanity_trades.copy(deep=True), prediction, russel=False)

In [ ]:
df_sanity_included, df_sanity_excluded = get_inclusion_exclusion_df(df_sanity_trades, prediction, russel=False)

print(df_sanity_included['profit_rel'].describe())
print(df_sanity_excluded['profit_rel'].describe())

# Feature Importance Analysis

## BB-Strategy

In [ ]:
# LGBM as it achieved the highest performance

# Load the data
# Train
df_train = load_ml_data(path_data_partitioned_bb_with_fees_train, 'v1_df_ml.csv', vertical_barrier = False)
df_train_features = df_train.filter(like='feature_')
df_train_target = df_train['bin']

# Test
df_test = load_ml_data(path_data_partitioned_bb_with_fees_test, 'v1_df_ho.csv', vertical_barrier = False)
df_test_features = df_test.filter(like='feature_')
df_test_target = df_test['bin']

# Config for LGBM (as determined by Optuna)
cfg = {
    "num_threads": 8,
    "boosting_type": "gbdt",
    "lambda_l1": 2.8269687100646643,
    "lambda_l2": 0.1132729746179371,
    "num_leaves": 222,
    "feature_fraction": 0.6029600980261194,
    "bagging_fraction": 0.9924346775670325,
    "bagging_freq": 2,
    "min_child_samples": 65,
    "max_depth": 14,
    "max_bin": 136,
    "learning_rate": 0.30197097330510875,
    "importance_type": "gain"
}

model_lgbm_final = lgb.LGBMClassifier(seed=42)
model_lgbm_final.set_params(**cfg)

fi = train_model_k_fold_feature_imp(model_lgbm_final, cfg, df_train_features, df_train_target)

fi['Feature'] = fi['Feature'].str.replace('feature_', '')
fi.head(20)

In [ ]:
# Visualize the feature importance
# Set custom feature names
custom_feature_names_gain = [
    'Normalized Average True Range (NATR): Period = 4  ',
    'NASDAQ Rolling Percentile Close Price  ',
    'GDP  ',
    'Fractional Differentiated Close Price  ',
    'Silver Close Price  ',
    'Treasury Yield 5 Years (^FVX)  ',
    'Generalized Autoregressive  \n Conditional Heteroskedasticity Close Price  ',
    'Gold Rolling Percentile Close Price  ',
    'Treasury Yield 10 Years (^TNX) 30-Day Moving Average  ',
    'Inflation Rolling Percentile  ',
]

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=fi.sort_values(by="Value", ascending=False)[:10], palette='viridis', 
            edgecolor='black', linewidth=1.5)

plt.suptitle('BB-Strategy - LGBM - 10 Most Important Features', fontsize=32)

plt.xticks(fontsize=20)
plt.yticks(range(10), custom_feature_names_gain, fontsize=24)

plt.xlabel('Importance', fontsize=30)
plt.ylabel('Feature', fontsize=30)

plt.subplots_adjust(top=0.9) 

plt.tight_layout()

# Save and show the plot
plt.savefig(path_default + '/visuals/bb_feature_importance_lgbm_gain.svg', dpi='figure', format='svg')
plt.show()

## TBM-Strategy

In [ ]:
# Load the data
# Train
df_train = load_ml_data(path_data_partitioned_tbm_with_fees_train, 'v1_df_ml.csv', vertical_barrier = True)
df_train_features = df_train.filter(like='feature_')
df_train_target = df_train['bin']

# Test
df_test = load_ml_data(path_data_partitioned_tbm_with_fees_test, 'v1_df_ho.csv', vertical_barrier = True)
df_test_features = df_test.filter(like='feature_')
df_test_target = df_test['bin']


# XGBoost achieved the highest performance
model_xgb = xgb.XGBClassifier(n_jobs=8)

# Config for XGBoost (as determined by Optuna)
cfg_xgb = {
    'lambda': 3.822976947706376,
    'alpha': 0.4445173954046871,
    'colsample_bytree': 0.6,
    'subsample': 0.6,
    'learning_rate': 0.02,
    'max_depth': 17,
    'min_child_weight': 13,
    'gamma': 0.3490986093851895,
    'grow_policy': 'lossguide'
}

model_xgb.set_params(**cfg_xgb)

fi = train_model_k_fold_feature_imp(model_xgb, cfg_xgb, df_train_features, df_train_target)

fi['Feature'] = fi['Feature'].str.replace('feature_', '')
fi.head(20)

In [ ]:
# Group and sort the feature importances
fi = fi.groupby('Feature')['Value'].mean().reset_index()
fi = fi.sort_values(by='Value', ascending=False).reset_index(drop=True)

# Custom feature names list for XGBoost
custom_feature_names_xgboost = [
    'S&P 500 New Lows: Period = 52 Days  ',
    'NASDAQ Rolling Percentile Close Price  ',
    'S&P 500 New Lows: Period = 52 Days; Rolling Percentile  ',
    'USD Index Rolling Percentile Close Price  ',
    'Crude Oil Rolling Percentile Close Price  ',
    'Crude Oil Close Price  ',
    'Normalized Average True Range (NATR): Period = 4  ',
    'USD Index Close Price  ',
    'Unemployment Rate  ',
    'Treasury Yield 3 Months (^IRX)  '
]

# Create the plot
plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=fi[:10], palette='viridis', 
            edgecolor='black', linewidth=1.5)

plt.xticks(fontsize=20)
plt.yticks(range(10), custom_feature_names_xgboost, fontsize=24)

plt.xlabel('Importance', fontsize=30)
plt.ylabel('Feature', fontsize=30)

plt.suptitle('TBM-Strategy - XGBoost - 10 Most Important Features', fontsize=32)
plt.subplots_adjust(top=0.9)  # Adjust the top parameter as needed

plt.tight_layout()

# Save and show the plot
plt.savefig(path_default + '/visuals/tbm_feature_importance_xgboost.svg', dpi='figure', format='svg')
plt.show()


# Exclusion Analysis

In [ ]:
# Define 10 most important features for BB-Strategy
feature_names_bb = [
    'feature_NATR',
    'feature_NASDAQ_rolling_percentile',
    'feature_GDP',
    'feature_frac_diff_close',
    'feature_Silver',
    'feature_Treasury_Yield3',
    'feature_garch_vol_close',
    'feature_Gold_rolling_percentile',
    'feature_treasury_yield1_ma30_rolling_percentile',
    'feature_Inflation_rolling_percentile'
]

# Define 10 most important features for TBM-Strategy
feature_names_tbm = [
    'feature_SP500_new_lows',
    'feature_NASDAQ_rolling_percentile',
    'feature_SP500_new_lows_rolling_percentile',
    'feature_USDIndex_rolling_percentile',
    'feature_CrudeOil_rolling_percentile',
    'feature_CrudeOil',
    'feature_NATR',
    'feature_USDIndex',
    'feature_Unemployment',
    'feature_Treasury_Yield2'
]

def inclusion_exclusion_analysis(inclusion, exclusion, feature_names, sides=False):
    """
    Prints the summary statistics for the given features for the validated and rejected trades.

    Args:
        inclusion (dataframe): DataFrame containing the validated trades.
        exclusion (dataframe): DataFrame containing the rejected trades.
        feature_names (list): List of feature names, depending on the 10 most important features.
        sides (boolean, optional): Whether to print the summary statistics for the two sides (-1 and 1). Defaults to False.
    """
    if sides: # If sides, long and short trades exist and should be considered separately
        side_1_inclusion = inclusion[inclusion['side'] == 1]
        side_1_exclusion = exclusion[exclusion['side'] == 1]
        side_minus_1_inclusion = inclusion[inclusion['side'] == -1]
        side_minus_1_exclusion = exclusion[exclusion['side'] == -1]
        
    for feature in feature_names:
        print('Inclusion Overall:')
        print(inclusion[feature].describe())

        print('Exclusion Overall:')
        print(exclusion[feature].describe())

        print('-'*50)

        if sides:
            print('Inclusion Side 1:')
            print(side_1_inclusion[feature].describe())
            print('Exclusion Side 1:')
            print(side_1_exclusion[feature].describe())

            print('-'*50)

            print('Inclusion Side -1:')
            print(side_minus_1_inclusion[feature].describe())
            print('Exclusion Side -1:')
            print(side_minus_1_exclusion[feature].describe())
            print('#'*50)

# Meta-Labeling w/ most promiment models

## Store Prediction CSVs

### BB-Strategy

In [ ]:
########## XGB LGBM ##########
# Train
df_train = load_ml_data(path_data_partitioned_bb_with_fees_train, 'v1_df_ml.csv', vertical_barrier = False)
df_train_features = df_train.filter(like='feature_')
df_train_target = df_train['bin']

# Test
df_test = load_ml_data(path_data_partitioned_bb_with_fees_test, 'v1_df_ho.csv', vertical_barrier = False)
df_test_features = df_test.filter(like='feature_')
df_test_target = df_test['bin']
########## XGB LGBM ##########


# Neural Network
df_train_features_nn = pd.read_csv(path_data_partitioned_bb_with_fees_scaled + '/v1_df_ml_features_scaled_ss.csv')
df_test_features_nn = pd.read_csv(path_data_partitioned_bb_with_fees_scaled + '/v1_df_ho_features_scaled_ss.csv')

test_dataset = CustomStockDataSet(df_test_features_nn, df_test_target)
test_dl = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

# Load the best model
path = '/home/lschiff_ext/data_v5/results/bb/neural_network_final_model/good'
model_name = 'epoch_80_2023-07-14-14-40-29'

full_path = path + '/' + model_name
model = CustomMLP_BB(df_train_features_nn.shape[1])
clf = load_model(model, full_path)

In [ ]:
# Create predictions for all best models XGBoost, LGBM, and the ANN

# XGBoost
bb_clf_xgb = load_model_sklearn(path_data_final_models, 'BB_XGBoost')
bb_pred_xgb = bb_clf_xgb.predict(df_test_features)
bb_pred_xgb_proba = bb_clf_xgb.predict_proba(df_test_features)

# LGBM
bb_clf_lgbm = load_model_sklearn(path_data_final_models, 'BB_LGBM')
bb_pred_lgbm = bb_clf_lgbm.predict(df_test_features)
bb_pred_lgbm_proba = bb_clf_lgbm.predict_proba(df_test_features)

# Neural Network
clf.eval()
predicted_labels_list = []
with torch.no_grad():
    for xb_test, yb_test in test_dl:                
        pred_test = clf(xb_test)
        threshold = 0.5
        predicted_labels = (pred_test >= threshold).int()

        predicted_labels_np = predicted_labels.cpu().numpy().flatten()  # or predicted_labels.squeeze()
        predicted_labels_list.append(predicted_labels_np)
pred_nn_bb = np.concatenate(predicted_labels_list)

prediction_majority_bb = majority_vote(bb_pred_xgb, bb_pred_lgbm, pred_nn_bb)
prediction_aggregation_bb = aggregation_vote(bb_pred_xgb, bb_pred_lgbm, pred_nn_bb)

# Save predictions for aggregation and majority vote
store_prediction_csv(prediction_majority_bb, 'BB_MAJORITY.csv')
store_prediction_csv(prediction_aggregation_bb, 'BB_AGGREGATION.csv')

# Save predictions and probability predictions
store_prediction_csv(bb_pred_xgb, 'BB_XGB.csv')
store_prediction_csv(bb_pred_xgb_proba, 'BB_XGB_PROBA.csv')
store_prediction_csv(bb_pred_lgbm, 'BB_LGBM.csv')
store_prediction_csv(bb_pred_lgbm_proba, 'BB_LGBM_PROBA.csv')
store_prediction_csv(pred_nn_bb, 'BB_NN.csv')

### TBM-Strategy

In [ ]:
# Get all data for the different machine learning algorithms

########## Dataset XGB ##########
# Train
df_train_xgb = load_ml_data(path_data_partitioned_tbm_with_fees_train, 'v1_df_ml.csv', vertical_barrier = True)
df_train_features_xgb = df_train_xgb.filter(like='feature_')
df_train_target_xgb = df_train_xgb['bin']

# Test
df_test_xgb = load_ml_data(path_data_partitioned_tbm_with_fees_test, 'v1_df_ho.csv', vertical_barrier = True)
df_test_features_xgb = df_test_xgb.filter(like='feature_')
df_test_target_xgb = df_test_xgb['bin']
########## Dataset XGB ##########

########## Dataset LGBM ##########
# Train
df_train_lgbm = load_ml_data(path_data_partitioned_tbm_with_fees_train, 'v1_df_ml.csv', vertical_barrier = True)
df_train_features_lgbm = pd.read_csv(path_data_partitioned_tbm_with_fees_scaled + '/v1_df_ml_features_scaled_ss.csv')
df_train_target_lgbm = df_train_lgbm['bin']

# Test
df_test_lgbm = load_ml_data(path_data_partitioned_tbm_with_fees_test, 'v1_df_ho.csv', vertical_barrier = True)
df_test_features_lgbm = pd.read_csv(path_data_partitioned_tbm_with_fees_scaled + '/v1_df_ho_features_scaled_ss.csv')
df_test_target_lgbm = df_test_lgbm['bin']
########## Dataset LGBM ##########

# Neural Network
df_train_features_nn = pd.read_csv(path_data_partitioned_tbm_with_fees_scaled + '/v1_df_ml_features_scaled_ss.csv')
df_test_features_nn = pd.read_csv(path_data_partitioned_tbm_with_fees_scaled + '/v1_df_ho_features_scaled_ss.csv')

test_dataset = CustomStockDataSet(df_test_features_nn, df_test_target_xgb)
test_dl = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

path = '/home/lschiff_ext/data_v5/results/tbm/neural_network_final_model'
model_name = 'epoch_250'

full_path = path + '/' + model_name
model = CustomMLP_TBM(df_train_features_nn.shape[1])
clf = load_model(model, full_path)


In [ ]:
# Make predictions

# XGBoost
tbm_clf_xgb = load_model_sklearn(path_data_final_models, 'TBM_XGBoost')

pred_xgb = tbm_clf_xgb.predict(df_test_features_xgb)

# LGBM
tbm_clf_lgbm = load_model_sklearn(path_data_final_models, 'TBM_LGBM')

pred_lgbm = tbm_clf_lgbm.predict(df_test_features_lgbm)

# Neural Network
clf.eval()

predicted_labels_list = []

with torch.no_grad():
    for xb_test, yb_test in test_dl:                
        pred_test = clf(xb_test)
        threshold = 0.5
        predicted_labels = (pred_test >= threshold).int()

        predicted_labels_np = predicted_labels.cpu().numpy().flatten()  # or predicted_labels.squeeze()
        predicted_labels_list.append(predicted_labels_np)
pred_nn = np.concatenate(predicted_labels_list)

In [ ]:
# Save predictions for XGBoost, LGBM, and the ANN
store_prediction_csv(pred_xgb, 'TBM_XGB.csv')
store_prediction_csv(pred_lgbm, 'TBM_LGBM.csv')
store_prediction_csv(pred_nn, 'TBM_NN.csv')

In [ ]:
# Majority and aggregation vote
prediction_majority = majority_vote(pred_xgb, pred_lgbm, pred_nn)
prediction_aggregation = aggregation_vote(pred_xgb, pred_lgbm, pred_nn)

store_prediction_csv(prediction_majority, 'TBM_MAJORITY.csv')
store_prediction_csv(prediction_aggregation, 'TBM_AGGREGATION.csv')

In [ ]:
# Save predicted probabilities
pred_xgb_proba = tbm_clf_xgb.predict_proba(df_test_features_xgb)
pred_lgbm_proba = tbm_clf_lgbm.predict_proba(df_test_features_lgbm)
store_prediction_csv(pred_xgb_proba, 'TBM_XGB_PROBA.csv')
store_prediction_csv(pred_lgbm_proba, 'TBM_LGBM_PROBA.csv')

## Evaluate Meta-Labeling

#### Load Data

In [ ]:
##BB##
# Train
df_train_bb = load_ml_data(path_data_partitioned_bb_with_fees_train, 'v1_df_ml.csv', vertical_barrier = False)
df_train_features_bb = df_train_bb.filter(like='feature_')
df_train_target_bb = df_train_bb['bin']

# Test
df_test_bb = load_ml_data(path_data_partitioned_bb_with_fees_test, 'v1_df_ho.csv', vertical_barrier = False)
df_test_features_bb = df_test_bb.filter(like='feature_')
df_test_target_bb = df_test_bb['bin']

##TBM##
# Train
df_train_tbm = load_ml_data(path_data_partitioned_tbm_with_fees_train, 'v1_df_ml.csv', vertical_barrier = True)
df_train_features_tbm = df_train_tbm.filter(like='feature_')
df_train_target_tbm = df_train_tbm['bin']

# Test
df_test_tbm = load_ml_data(path_data_partitioned_tbm_with_fees_test, 'v1_df_ho.csv', vertical_barrier = True)
df_test_features_tbm = df_test_tbm.filter(like='feature_')
df_test_target_tbm = df_test_tbm['bin']

### Basic Evaluation

#### BB-Strategy

In [ ]:
# Get predictions with default threshold
prediction_lgbm_bb = load_prediction_csv('BB_LGBM_0.5.csv')
prediction_xgb_bb = load_prediction_csv('BB_XGB_0.5.csv')
prediction_nn_bb = load_prediction_csv('BB_NN_0.5.csv')

In [ ]:
# Model to chose from
prediction_bb = prediction_lgbm_bb
# prediction_bb = prediction_xgb_bb
# prediction_bb = prediction_nn_bb

calc_full_evaluation_w_prediction(prediction_bb, df_test_bb, df_test_features_bb, df_test_target_bb)

In [ ]:
full_financial_performance_eval(df_test_bb, prediction_lgbm_bb, fraction=False, russel=True)
full_financial_performance_eval(df_test_bb, prediction_lgbm_bb, fraction=False, russel=False)

#### TBM-Strategy

In [ ]:
# Get predictions with default threshold

prediction_xgb_tbm= load_prediction_csv('TBM_XGB_0.5.csv')
prediction_lgbm_tbm= load_prediction_csv('TBM_LGBM_0.5.csv')
prediction_nn_tbm= load_prediction_csv('TBM_NN_0.5.csv')

In [ ]:
# Model to chose from
prediction_tbm = prediction_xgb_tbm
# prediction_tbm = prediction_lgbm_tbm
# prediction_tbm = prediction_nn_tbm

calc_full_evaluation_w_prediction(prediction_tbm, df_test_tbm, df_test_features_tbm, df_test_target_tbm)

In [ ]:
full_financial_performance_eval(df_test_tbm, prediction_xgb_tbm, fraction=False, russel=True)
full_financial_performance_eval(df_test_tbm, prediction_xgb_tbm, fraction=False, russel=False)

### Trader Types (heavy-, medium-, low-cautious)

#### BB-Strategy

In [ ]:
# Load predicted probability for LGBM
pred_proba_LGBM_bb = load_prediction_csv('BB_LGBM_PROBA.csv', proba=True)

# Get trades based on thresholds
betsizing_prediction_06 = label_predictions(pred_proba_LGBM_bb, 0.6)
betsizing_prediction_07 = label_predictions(pred_proba_LGBM_bb, 0.7)
betsizing_prediction_08 = label_predictions(pred_proba_LGBM_bb, 0.8)

# Financial performance evaluation
full_financial_performance_eval(df_test_bb, betsizing_prediction_06, fraction=True, russel=True)
full_financial_performance_eval(df_test_bb, betsizing_prediction_06, fraction=True, russel=False)

full_financial_performance_eval(df_test_bb, betsizing_prediction_07, fraction=True, russel=True)
full_financial_performance_eval(df_test_bb, betsizing_prediction_07, fraction=True, russel=False)

full_financial_performance_eval(df_test_bb, betsizing_prediction_08, fraction=True, russel=True)
full_financial_performance_eval(df_test_bb, betsizing_prediction_08, fraction=True, russel=False)

#### TBM-Strategy

In [ ]:
# Load predicted probability for XGBoost
pred_proba_XGB_tbm = load_prediction_csv('TBM_XGB_PROBA.csv', proba=True)

# Get trades based on thresholds
betsizing_prediction_06_tbm = label_predictions(pred_proba_XGB_tbm, 0.6)
betsizing_prediction_07_tbm = label_predictions(pred_proba_XGB_tbm, 0.7)

# Financial performance evaluation
full_financial_performance_eval(df_test_tbm, betsizing_prediction_06_tbm, fraction=True, russel=True)
full_financial_performance_eval(df_test_tbm, betsizing_prediction_06_tbm, fraction=True, russel=False)

full_financial_performance_eval(df_test_tbm, betsizing_prediction_07_tbm, fraction=True, russel=True)
full_financial_performance_eval(df_test_tbm, betsizing_prediction_07_tbm, fraction=True, russel=False)

### Fractional Trader

#### BB-Strategy

In [ ]:
# Load predicted probability for LGBM
pred_proba_LGBM_bb = load_prediction_csv('BB_LGBM_PROBA.csv', proba=True)

# Fractional investments
betsizing_prediction_abs_frac_bb = label_predictions_fraction(pred_proba_LGBM_bb, 0.5, 0.9)

# Evaluation
full_financial_performance_eval(df_test_bb.copy(deep=True), betsizing_prediction_abs_frac_bb, fraction=True, russel=False)
full_financial_performance_eval(df_test_bb.copy(deep=True), betsizing_prediction_abs_frac_bb, fraction=True, russel=True)

#### TBM-Strategy

In [ ]:
# Load predicted probability for XGBoost
pred_proba_tbm_xgb = load_prediction_csv('TBM_XGB_PROBA.csv', proba=True)

# Fractional investments
betsizing_prediction_abs_frac_tbm = label_predictions_fraction(pred_proba_tbm_xgb, 0.5, 0.68)

# Evaluation
full_financial_performance_eval(df_test_tbm.copy(deep=True), betsizing_prediction_abs_frac_tbm, fraction=True, russel=False)
full_financial_performance_eval(df_test_tbm.copy(deep=True), betsizing_prediction_abs_frac_tbm, fraction=True, russel=True)

### Threshold Optimization


#### BB-Strategy

In [ ]:
# Create train and test sets 
df_train_before_2017 = df_train_bb[df_train_bb['date'] <= pd.to_datetime('2017-01-01')]
df_train_after_2017 = df_train_bb[df_train_bb['date'] > pd.to_datetime('2017-01-01')]

# Data for training the machine learning model
df_train_before_2017_features = df_train_before_2017.filter(like='feature_')
df_train_before_2017_target = df_train_before_2017['bin']

# Data for determining the optimal threshold
df_train_after_2017_features = df_train_after_2017.filter(like='feature_')
df_train_after_2017_target = df_train_after_2017['bin']

print(len(df_train_before_2017))
print(len(df_train_after_2017))
print('Size Training:', 1 - (len(df_train_after_2017) / len(df_train_before_2017)))

In [ ]:
# Only Train LGBM as it is the most prominent model
config_part_lgbm = {
    "num_threads": 8,
    "boosting_type": "gbdt",
    "lambda_l1": 2.8269687100646643,
    "lambda_l2": 0.1132729746179371,
    "num_leaves": 222,
    "feature_fraction": 0.6029600980261194,
    "bagging_fraction": 0.9924346775670325,
    "bagging_freq": 2,
    "min_child_samples": 65,
    "max_depth": 14,
    "max_bin": 136,
    "learning_rate": 0.30197097330510875
}

betsizing_part = lgb.LGBMClassifier(seed=42)
betsizing_part_trained = train_model(betsizing_part, config_part_lgbm, df_train_before_2017_features, df_train_before_2017_target)

In [ ]:
betsizing_part_pred = betsizing_part_trained.predict_proba(df_train_after_2017_features)

In [ ]:
determine_and_visualize_optimal_threshold(betsizing_part_pred, df_train_after_2017_target)

#### TBM-Strategy

In [ ]:
# Create train and test sets 
df_train_before_2017_tbm = df_train_tbm[df_train_tbm['date'] <= pd.to_datetime('2017-01-01')]
df_train_after_2017_tbm = df_train_tbm[df_train_tbm['date'] > pd.to_datetime('2017-01-01')]

# Data for training the machine learning model
df_train_before_2017_tbm_features = df_train_before_2017_tbm.filter(like='feature_')
df_train_before_2017_tbm_target = df_train_before_2017_tbm['bin']

# Data for determining the optimal threshold
df_train_after_2017_tbm_features = df_train_after_2017_tbm.filter(like='feature_')
df_train_after_2017_tbm_target = df_train_after_2017_tbm['bin']

print(len(df_train_before_2017_tbm))
print(len(df_train_after_2017_tbm))
print('Size Training:', 1 - (len(df_train_after_2017_tbm) / len(df_train_before_2017_tbm)))

In [ ]:
config_part_xgb_tbm = {
    'lambda': 3.822976947706376,
    'alpha': 0.4445173954046871,
    'colsample_bytree': 0.6,
    'subsample': 0.6,
    'learning_rate': 0.02,
    'max_depth': 17,
    'min_child_weight': 13,
    'gamma': 0.3490986093851895,
    'grow_policy': 'lossguide'
}

betsizing_part_tbm = xgb.XGBClassifier(n_jobs=8)
betsizing_part_trained_tbm = train_model(betsizing_part_tbm, config_part_xgb_tbm, df_train_before_2017_tbm_features, df_train_before_2017_tbm_target)

In [ ]:
betsizing_part_pred_tbm = betsizing_part_trained_tbm.predict_proba(df_train_after_2017_tbm_features)

In [ ]:
determine_and_visualize_optimal_threshold(betsizing_part_pred_tbm, df_train_after_2017_tbm_target, 'TBM_OPT_THRESHOLD.png')

### Visualize Performance Meta-Labeling

In [ ]:
# Load predictions
prediction_loaded_bb = load_prediction_csv('BB_LGBM_0.9.csv')
prediction_loaded_tbm = load_prediction_csv('TBM_XGB_0.6.csv')

# Visualize the performance of Meta-Labeling
visualize_performance_meta_labeling(df_test_bb, prediction_loaded_bb, show_exclusion=True, bb=True)
visualize_performance_meta_labeling(df_test_tbm, prediction_loaded_tbm, show_exclusion=True, bb=False)